#FASE DE PREPARACIÓN



## Instalación de Bibliotecas


In [1]:
# --- PASO 1: INSTALAR BIBLIOTECAS NECESARIAS ---
print("Instalando bibliotecas necesarias (streamlit, pyngrok, google-cloud-bigquery)...")
!pip install streamlit pyngrok google-cloud-bigquery -q
print("✓ Instalación de bibliotecas completada.")
print("-" * 50)

!pip install streamlit pandas google-cloud-bigquery
!pip install streamlit-quill

Instalando bibliotecas necesarias (streamlit, pyngrok, google-cloud-bigquery)...
✓ Instalación de bibliotecas completada.
--------------------------------------------------


## Configuración y Verificación de Credenciales


In [2]:
# --- PASO 2: VERIFICACIÓN DEL ARCHIVO DE CREDENCIALES ---
import os
from google.colab import files

CREDENTIALS_FILE = "/content/miespacioterapeutico-1283652e341e.json"

print(f"Verificando la existencia del archivo de credenciales en: {CREDENTIALS_FILE}")

# Si el archivo no existe, pide al usuario que lo suba
if not os.path.exists(CREDENTIALS_FILE):
    print("\n⚠️ Archivo de credenciales no encontrado. Por favor, sube el archivo 'miespacioterapeutico-1283652e341e.json' ahora.")
    uploaded = files.upload()

    # Intenta renombrar el archivo si se subió con un nombre diferente
    for fn in uploaded.keys():
        if fn != CREDENTIALS_FILE.split('/')[-1]:
            try:
                os.rename(f"/content/{fn}", CREDENTIALS_FILE)
                print(f"✓ Archivo subido renombrado a: '{CREDENTIALS_FILE.split('/')[-1]}'")
            except Exception as e:
                print(f"❌ No se pudo renombrar el archivo subido: {e}")
        else:
            print(f"✓ Archivo subido: '{fn}'")

# Volver a verificar la existencia después de la subida
if os.path.exists(CREDENTIALS_FILE):
    print("✅ El archivo de credenciales fue encontrado y está listo.")
else:
    print("❌ Error: El archivo de credenciales NO fue encontrado en la ruta especificada. La ejecución no continuará.")
    raise FileNotFoundError("Credenciales de BigQuery no encontradas.")

print("-" * 50)

Verificando la existencia del archivo de credenciales en: /content/miespacioterapeutico-1283652e341e.json
✅ El archivo de credenciales fue encontrado y está listo.
--------------------------------------------------


## Configuración del Token de ngrok


In [3]:
# --- PASO 3: CONFIGURAR TOKEN DE NGROK ---
from pyngrok import ngrok
import getpass
import os

NGROK_AUTH_TOKEN = "32ErhfVTbh91cHC38eX9mb76hIv_5gYyLQ5cZJbVvHJA79mZK" # Reemplaza con tu token real

print("Configurando token de ngrok...")
try:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("✓ Token de ngrok configurado.")
except Exception as e:
    print(f"⚠️ Error al configurar el token de ngrok: {e}")
print("-" * 50)

Configurando token de ngrok...
✓ Token de ngrok configurado.
--------------------------------------------------


#FASE IMPLEMENTACIÓN DE LA APLICACIÓN

## Imports y Creación del Archivo


In [4]:
# --- PASO 1: IMPORTS Y CONFIGURACIÓN INICIAL ---
import streamlit as st
import uuid
import os
import datetime
import json
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2.service_account import Credentials
from datetime import timedelta

# --- CONFIGURACIÓN DE PÁGINA ---
st.set_page_config(
    page_title="MiEspacioTerapeutico",
    page_icon="🧠",
    layout="wide",
    initial_sidebar_state="expanded"
)

2025-09-18 21:18:34.578 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


## Configuración de BigQuery y Cliente


In [5]:
%%writefile -a app.py
# --- PASO 2: CONFIGURACIÓN DE BIGQUERY ---
PROJECT_ID = "miespacioterapeutico"
DATASET_ID = "Aplicacion_MiEspacioTerapeutico"
CREDENTIALS_FILE = "miespacioterapeutico-1283652e341e.json"

if not os.path.exists(CREDENTIALS_FILE):
    st.error(f"Error: Archivo de credenciales NO encontrado: '{CREDENTIALS_FILE}'")
    st.stop()

@st.cache_resource
def get_bigquery_client():
    try:
        credentials = Credentials.from_service_account_file(CREDENTIALS_FILE)
        return bigquery.Client(credentials=credentials, project=PROJECT_ID)
    except Exception as e:
        st.error(f"Error de autenticación con BigQuery: {e}")
        return None

client = get_bigquery_client()
schemas = get_table_schemas(client)

Appending to app.py


## Gestión del Estado de la Sesión (Session State)

In [6]:
%%writefile -a app.py
# --- PASO 3: GESTIÓN DE ESTADO (SESSION STATE) ---
def initialize_session_state():
    '''Inicializa todas las listas y claves necesarias en el estado de la sesión.'''
    keys_to_initialize = {
        'current_patient_id': None, 'hijos_list': [],
        'antecedentes_medicos_list': [], 'antecedentes_psiquiatricos_list': [],
        'familiares_list': [], 'selected_msea_items': [], 'current_patient_diagnoses': [],
        'tratamientos_farmacologicos_list': [], 'tratamientos_psiquiatricos_list': [],
        'sesiones_list': [], 'agenda_list': [], 'examen_mental_redaccion': '', 'observaciones_generales': '', 'msea_categoria_sel': '', 'msea_subcat_sel': '',
        'datos_administrativos': {}, 'motivo_de_consulta': {}, 'examen_mental': {},
        'antecedentes_personales': {}, 'antecedentes_familiares': {},
        'habitos_psicobiologicos': {}, 'diagnosticos': {}, 'tratamientos': {}, 'comprension_psicodinamica': {}
    }
    for key, value in keys_to_initialize.items():
        if key not in st.session_state:
            st.session_state[key] = value

initialize_session_state()

Appending to app.py


## Funciones de manejo de datos y lógica auxiliar


In [7]:
# --- FUNCIONES DE MANEJO DE DATOS Y LÓGICA AUXILIAR ---
def safe_date_converter(value):
    if pd.isna(value) or value is None: return None
    if isinstance(value, datetime.datetime): return value.date()
    if isinstance(value, datetime.date): return value
    try: return pd.to_datetime(value).date()
    except: return None

def safe_time_converter(value):
    if value is None or pd.isna(value): return None
    if isinstance(value, datetime.time): return value
    try: return pd.to_datetime(value).time()
    except: return None

def safe_decimal_converter(value, default=None):
    if value is None or value == '':
        return default
    try:
        return Decimal(str(value))
    except InvalidOperation:
        return default

def generar_narrativa_examen(selected_items):
    """
    Genera un texto narrativo coherente a partir de los ítems del examen mental,
    agrupándolos por categoría para una mejor lectura clínica.
    """
    if not selected_items:
        return "Paciente sin alteraciones significativas en las áreas evaluadas del examen mental."

    texto_por_categoria = {}
    for item in selected_items:
        cat = item['categoria']
        sub = item['subcategoria']
        nota = item.get('notas', '').strip()

        if cat not in texto_por_categoria:
            texto_por_categoria[cat] = []

        frase = sub
        if nota:
            frase += f": {nota}"
        texto_por_categoria[cat].append(frase)

    narrativa_final = []
    for cat, frases in texto_por_categoria.items():
        narrativa_final.append(f"En el área de {cat.lower()}, se observa: {', '.join(frases)}.")

    return " ".join(narrativa_final)

def upsert_data(table_name, data_dict):
    primary_key_map = {'sesiones': 'id_sesion', 'agenda': 'id_cita'}
    primary_key = primary_key_map.get(table_name, 'id_paciente')
    if not client or not data_dict.get(primary_key): return False

    temp_table_id = None
    try:
        table_id = f"{PROJECT_ID}.{DATASET_ID}.{table_name}"
        table = table_objects[table_name]
        schema_cols = [field.name for field in table.schema]

        cleaned_data = {k: v for k, v in data_dict.items() if k in schema_cols}

        df = pd.DataFrame([cleaned_data])

        for field in table.schema:
            col_name = field.name
            if col_name in df.columns:
                if field.field_type == 'TIME':
                    df[col_name] = df[col_name].apply(
                        lambda t: t.strftime('%H:%M:%S') if pd.notna(t) and isinstance(t, datetime.time) else None
                    )
                elif field.field_type == 'DATE':
                     df[col_name] = pd.to_datetime(df[col_name], errors='coerce').dt.date

        job_config = bigquery.LoadJobConfig(schema=table.schema)
        temp_table_id = f"{PROJECT_ID}.{DATASET_ID}.temp_{table_name}_{generate_unique_id()}"
        load_job = client.load_table_from_dataframe(df, temp_table_id, job_config=job_config)
        load_job.result()

        update_cols = [f"T.`{col}` = S.`{col}`" for col in df.columns if col != primary_key]
        insert_cols = [f"`{col}`" for col in df.columns]
        source_cols = [f"S.`{col}`" for col in df.columns]

        merge_sql = f"""MERGE `{table_id}` T USING `{temp_table_id}` S ON T.`{primary_key}` = S.`{primary_key}`
                            WHEN MATCHED THEN UPDATE SET {', '.join(update_cols)}
                            WHEN NOT MATCHED THEN INSERT ({', '.join(insert_cols)}) VALUES ({', '.join(source_cols)})"""

        if not update_cols:
            merge_sql = f"""MERGE `{table_id}` T USING `{temp_table_id}` S ON T.`{primary_key}` = S.`{primary_key}`
                                WHEN NOT MATCHED THEN INSERT ({', '.join(insert_cols)}) VALUES ({', '.join(source_cols)})"""

        client.query(merge_sql).result()
        return True
    except Exception as e:
        st.error(f"Error al guardar en '{table_name}': {e}"); return False
    finally:
        if temp_table_id: client.delete_table(temp_table_id, not_found_ok=True)

def delete_patient(patient_id):
    if not client or not patient_id: return False
    tables_to_delete_from = [
        'agenda', 'sesiones', 'comprension_psicodinamica', 'tratamientos',
        'diagnosticos', 'examen_mental', 'habitos_psicobiologicos',
        'antecedentes_familiares', 'antecedentes_personales', 'enfermedad_actual',
        'motivo_de_consulta', 'datos_administrativos', 'datos_identificacion',
        'pacientes'
    ]
    try:
        for table_name in tables_to_delete_from:
            query = f"DELETE FROM `{PROJECT_ID}.{DATASET_ID}.{table_name}` WHERE id_paciente = @patient_id"
            job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ScalarQueryParameter("patient_id", "STRING", patient_id)])
            client.query(query, job_config=job_config).result()
        st.cache_data.clear()
        st.success("Paciente y todos sus datos han sido eliminados permanentemente.")
        return True
    except Exception as e:
        st.error(f"Error al eliminar el paciente: {e}"); return False

def delete_record(table_name, record_id, id_column):
    if not client or not record_id: return False
    try:
        query = f"DELETE FROM `{PROJECT_ID}.{DATASET_ID}.{table_name}` WHERE {id_column} = @record_id"
        job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ScalarQueryParameter("record_id", "STRING", record_id)])
        client.query(query, job_config=job_config).result()
        st.success(f"Registro eliminado de la tabla '{table_name}'.")
        return True
    except Exception as e:
        st.error(f"Error al eliminar el registro: {e}"); return False

@st.cache_data(ttl=300)
def get_patient_data_from_db(_client, _table_objects_dict, patient_id):
    if not _client or not patient_id or not _table_objects_dict: return {}
    data = {}
    table_schemas = {name: [field.name for field in table.schema] for name, table in _table_objects_dict.items()}
    tables_to_join = [
        'datos_identificacion', 'datos_administrativos', 'motivo_de_consulta','enfermedad_actual',
        'antecedentes_personales', 'antecedentes_familiares', 'habitos_psicobiologicos',
        'examen_mental', 'diagnosticos', 'tratamientos', 'comprension_psicodinamica'
    ]
    joins, select_aliases = "", []
    for table_name in tables_to_join:
        alias = ''.join([part[0] for part in table_name.split('_')])
        joins += f"LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.{table_name}` AS {alias} ON p.id_paciente = {alias}.id_paciente \n"
        select_aliases.append(f"{alias}.*")
    main_query = f"SELECT p.*, {', '.join(select_aliases)} FROM `{PROJECT_ID}.{DATASET_ID}.pacientes` AS p {joins} WHERE p.id_paciente = @patient_id LIMIT 1"
    job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ScalarQueryParameter("patient_id", "STRING", patient_id)])
    try:
        df_main = _client.query(main_query, job_config=job_config).to_dataframe()
        all_tables = ['pacientes'] + tables_to_join
        for table in all_tables: data[table] = {}
        if not df_main.empty:
            main_data_row = df_main.to_dict('records')[0]
            for col, value in main_data_row.items():
                if pd.notna(value):
                    for tbl in all_tables:
                        if col in table_schemas.get(tbl, []):
                            data[tbl][col] = value
                            break
    except Exception as e:
        st.error(f"Error al cargar datos principales del paciente: {e}"); return {}
    for table_name in ['sesiones', 'agenda']:
        try:
            query = f"SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.{table_name}` WHERE id_paciente = @patient_id"
            job_config_many = bigquery.QueryJobConfig(query_parameters=[bigquery.ScalarQueryParameter("patient_id", "STRING", patient_id)])
            df = _client.query(query, job_config_many).to_dataframe()
            data[table_name] = df.to_dict('records') if not df.empty else []
        except: data[table_name] = []
    return data

def on_select_patient(patient_id):
    st.session_state.current_patient_id = patient_id
    if patient_id:
        with st.spinner("Cargando datos del paciente..."):
            data = get_patient_data_from_db(client, table_objects, patient_id)
            for table, record in data.items():
                if table not in ['sesiones', 'agenda']: st.session_state[table] = record or {}
            st.session_state.hijos_list = list(st.session_state.get('datos_identificacion', {}).get('hijos', []) or [])
            st.session_state.antecedentes_medicos_list = list(st.session_state.get('antecedentes_personales', {}).get('antecedentes_medicos', []) or [])
            st.session_state.antecedentes_psiquiatricos_list = list(st.session_state.get('antecedentes_personales', {}).get('antecedentes_psiquiatricos', []) or [])
            st.session_state.familiares_list = list(st.session_state.get('antecedentes_familiares', {}).get('familiares', []) or [])
            st.session_state.selected_msea_items = list(st.session_state.get('examen_mental', {}).get('msea_items', []) or [])
            st.session_state.current_patient_diagnoses = list(st.session_state.get('diagnosticos', {}).get('diagnosticos', []) or [])
            st.session_state.tratamientos_farmacologicos_list = list(st.session_state.get('tratamientos', {}).get('farmacologico', []) or [])
            st.session_state.tratamientos_psiquiatricos_list = list(st.session_state.get('tratamientos', {}).get('psiquiatrico', []) or [])
            st.session_state.sesiones_list = data.get('sesiones', [])
            st.session_state.agenda_list = data.get('agenda', [])
    st.session_state.page = 'ficha'
    st.rerun()

@st.cache_data(ttl=3600)
def load_reference_data(_client):
    if not _client: return {}, [], [], {}, [], {}
    try:
        df_paises = client.query(f"SELECT country_name, country_id FROM `{PROJECT_ID}.{DATASET_ID}.Paises`").to_dataframe()
        country_options = [''] + sorted(df_paises['country_name'].unique().tolist())
        cities_by_country = {country: [''] + sorted(df_paises[df_paises['country_name'] == country]['country_id'].tolist()) for country in df_paises['country_name'].unique()}
        df_em = client.query(f"SELECT Categoria, `Sub- categoria` FROM `{PROJECT_ID}.{DATASET_ID}.Examen_mental`").to_dataframe()
        df_em.columns = ['Categoria', 'Sub_categoria']
        msea_data = {cat: sorted(group['Sub_categoria'].tolist()) for cat, group in df_em.groupby('Categoria')}
        msea_order = ['Aspecto','Vestimenta','Biotipo','Actitud','Nivel de conciencia','Estado de conciencia','Orientación','Atención','Memoria','Lenguaje','Afectividad','Inteligencia','Velocidad del pensamiento','Curso del pensamiento','Tipo del pensamiento','Contenido de pensamiento','Sensopercepción','Juicio de realidad','Psicomotricidad','Conciencia de enfermedad']
        df_diag = client.query(f"SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.Diagnostico`").to_dataframe()
        diag_categories = sorted(list(set(df_diag['Categoría'].dropna())))
        grouped_diag_data = df_diag.groupby(['Categoría', 'Subcategoría']).apply(lambda x: x.to_dict('records')).to_dict()
        return msea_data, msea_order, diag_categories, grouped_diag_data, country_options, cities_by_country
    except Exception as e:
        st.error(f"Error al cargar datos de referencia: {e}"); return {}, [], [], {}, [], {}

2025-09-18 21:18:34.681 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-18 21:18:34.687 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager



## Carga de datos de referencia e inicio de UI

In [8]:
%%writefile -a app.py
# --- PASO 5: CARGA DE DATOS DE REFERENCIA E INICIO DE UI ---
organized_msea_data, msea_category_order, diag_categories, grouped_diag_data, country_options, cities_by_country = load_reference_data(client)

st.title("MiEspacioTerapeutico")
st.markdown("Una herramienta para la gestión profesional de expedientes.")
st.markdown("---")

Appending to app.py


## Interfaz de Usuario - Barra Lateral (Sidebar)




---


## Interfaz de Usuario - Área Principal y Pestañas


In [9]:
%%writefile app.py
# --- PASO 1: IMPORTS Y CONFIGURACIÓN INICIAL ---
import streamlit as st
import os
import datetime
import json
import pandas as pd
import numpy as np
import time
import re  # 👈 para limpiar HTML en narrativa
import re
import unicodedata
from google.cloud import bigquery
from google.oauth2.service_account import Credentials
from datetime import timedelta
from streamlit_quill import st_quill
from decimal import Decimal, InvalidOperation

# --- FUNCIÓN PARA GENERAR IDs ÚNICOS ---
def generate_unique_id():
    """Genera un ID único basado en timestamp y aleatoriedad, evitando el formato UUID."""
    timestamp = datetime.datetime.utcnow().strftime('%Y%m%d%H%M%S%f')
    random_part = os.urandom(4).hex()
    return f"{timestamp}_{random_part}"

# --- CONFIGURACIÓN DE PÁGINA ---
st.set_page_config(
    page_title="MiEspacioTerapeutico",
    page_icon="🧠",
    layout="wide",
    initial_sidebar_state="expanded"
)

# --- ESTILOS CSS Y CABECERA PERSONALIZADA ---
st.markdown("""
    <style>
        .main-header {
            background-color: #ffffff;
            padding: 0.5rem 1rem;
            border-bottom: 1px solid #e0e0e0; font-size: 24px;
            font-weight: 600; color: #31333F; position: fixed;
            top: 0; left: 0; width: 100%; z-index: 1000;
            text-align: center;
            box-shadow: 0 2px 4px rgba(0,0,0,0.05);
        }
        .main > div { padding-top: 5rem; }
        header { visibility: hidden; height: 0px; margin-top: -50px; }
        .sidebar-title {
            font-size: 22px;
            font-weight: 600;
            text-align: center;
            margin-bottom: 1rem;
            color: #31333F;
        }
        .page-title {
            font-size: 28px;
            font-weight: 600;
        }
        .patient-name-header {
            font-size: 24px;
            font-weight: 600;
            color: #31333F;
        }
        .column-header {
            color: #0072B2;
            font-weight: 600;
        }
        [data-baseweb="tab-list"] {
            background-color: #F0F2F6;
            border-radius: 8px;
            padding: 4px;
            display: flex;
            width: 100%;
        }
        [data-baseweb="tab"] {
            flex-grow: 1;
            justify-content: center;
            color: #31333F;
            font-weight: 600;
            border-radius: 6px;
        }
        [data-baseweb="tab"][aria-selected="true"] {
            color: #FFFFFF;
            background-color: #FF4B4B;
        }
    </style>
    <div class="main-header">
        MiEspacioTerapeutico
    </div>
""", unsafe_allow_html=True)

# --- CONFIGURACIÓN DE BIGQUERY Y SCHEMAS ---
PROJECT_ID = "miespacioterapeutico"
DATASET_ID = "Aplicacion_MiEspacioTerapeutico"
CREDENTIALS_FILE = "miespacioterapeutico-1283652e341e.json"

@st.cache_resource
def get_bigquery_client():
    if not os.path.exists(CREDENTIALS_FILE):
        st.error(f"Error Crítico: No se encuentra el archivo de credenciales '{CREDENTIALS_FILE}'."); st.stop()
    try:
        credentials = Credentials.from_service_account_file(CREDENTIALS_FILE)
        return bigquery.Client(credentials=credentials, project=PROJECT_ID)
    except Exception as e:
        st.error(f"Error de autenticación con BigQuery: {e}"); st.stop()
client = get_bigquery_client()

@st.cache_resource(ttl=3600)
def get_table_schemas(_client):
    if not _client: return {}
    table_names = [
        'pacientes', 'datos_identificacion', 'datos_administrativos', 'motivo_de_consulta',
        'enfermedad_actual', 'antecedentes_personales', 'antecedentes_familiares',
        'habitos_psicobiologicos', 'examen_mental', 'diagnosticos',
        'tratamientos', 'comprension_psicodinamica', 'sesiones', 'agenda'
    ]
    schemas = {}
    try:
        for table in table_names:
            table_ref = f"{PROJECT_ID}.{DATASET_ID}.{table}"
            schemas[table] = client.get_table(table_ref)
        return schemas
    except Exception as e:
        st.error(f"Error crítico al cargar los esquemas de las tablas: {e}"); return {}
table_objects = get_table_schemas(client)

# --- GESTIÓN DE ESTADO (SESSION STATE) ---
def initialize_session_state():
    defaults = {
        'page': 'consultorio', 'welcome_seen': False, 'current_patient_id': None,
        'loaded_patient_id': None,
        'hijos_list': [], 'antecedentes_medicos_list': [], 'antecedentes_psiquiatricos_list': [],
        'familiares_list': [], 'selected_msea_items': [], 'current_patient_diagnoses': [],
        'tratamientos_farmacologicos_list': [], 'tratamientos_psiquiatricos_list': [],
        'sesiones_list': [], 'agenda_list': []
    }
    for key, value in defaults.items():
        if key not in st.session_state: st.session_state[key] = value
initialize_session_state()

# --- FUNCIONES DE MANEJO DE DATOS Y LÓGICA AUXILIAR ---
def safe_date_converter(value):
    if pd.isna(value) or value is None: return None
    if isinstance(value, datetime.datetime): return value.date()
    if isinstance(value, datetime.date): return value
    try: return pd.to_datetime(value).date()
    except: return None

def safe_time_converter(value):
    if value is None or pd.isna(value): return None
    if isinstance(value, datetime.time): return value
    try: return pd.to_datetime(value).time()
    except: return None

def safe_decimal_converter(value, default=None):
    if value is None or value == '':
        return default
    try:
        return Decimal(str(value))
    except InvalidOperation:
        return default

def generar_narrativa_examen(selected_items):
    if not selected_items:
        return "<p>Paciente sin alteraciones significativas en las áreas evaluadas del examen mental.</p>"

    texto_por_categoria = {}
    for item in selected_items:
        cat = item['categoria']
        sub = item['subcategoria']
        nota = item.get('notas', '').strip()

        if cat not in texto_por_categoria:
            texto_por_categoria[cat] = []

        frase = f"<strong>{sub}</strong>"
        if nota:
            frase += f": {nota}"
        texto_por_categoria[cat].append(frase)

    narrativa_final = []
    for cat, frases in texto_por_categoria.items():
        narrativa_final.append(f"<p>En el área de <strong>{cat.lower()}</strong>, se observa: {', '.join(frases)}.</p>")

    return "".join(narrativa_final)

# ====== Narrativa V3 (texto plano, legible) con concordancia de género y normalización ======

# --- Helpers para narrativa clínica (mejorados) ---
import re
import unicodedata

def _strip_html_tags(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    s = re.sub(r"<[^>]+>", "", s)
    return s.strip()

def _remove_accents(s: str) -> str:
    s = "" if s is None else str(s)
    s = s.strip()
    s = unicodedata.normalize("NFKD", s)
    return "".join(ch for ch in s if not unicodedata.combining(ch))

def _canonize_display(s: str) -> str:
    """
    Normaliza a minúsculas sin acentos para reconocer subcategorías aunque
    vengan con mayúsculas o acentos distintos. Devuelve la clave “canónica”.
    """
    base = _remove_accents(s).lower()
    base = re.sub(r"\s+", " ", base)
    base = base.strip()

    CANON = {
        # Vestimenta
        "acorde a edad y contexto": "vestimenta_acorde",
        "acorde a edad, no acorde a contexto": "vestimenta_no_contexto",
        "desacorde a edad y contexto": "vestimenta_desacorde",

        # Biotipo
        "piqnico": "biotipo_picnico",  # por si llega mal escrito
        "pícnico": "biotipo_picnico",
        "astenico": "biotipo_astenico",
        "asténico": "biotipo_astenico",
        "atletico": "biotipo_atletico",
        "atlético": "biotipo_atletico",
        "displasico": "biotipo_displasico",
        "displásico": "biotipo_displasico",
        "ectomorfo (delgado, alargado, poca masa muscular)": "biotipo_ectomorfo",
        "mesomorfo (musculoso, estructura osea fuerte)": "biotipo_mesomorfo",
        "mesomorfo (musculoso, estructura ósea fuerte)": "biotipo_mesomorfo",
        "endomorfo (predominio de grasa, cuerpo redondeado)": "biotipo_endomorfo",

        # Actitud (algunas más comunes)
        "colaboradora": "actitud_colaboradora",
        "indiferente": "actitud_indiferente",
        "hostil": "actitud_hostil",
        "suspicaz": "actitud_suspicaz",
        "negativa": "actitud_negativa",
        "de rechazo": "actitud_rechazo",
        "defensiva": "actitud_defensiva",
        "oposicionista": "actitud_oposicionista",
        "desafiante": "actitud_desafiante",
        "sumisa": "actitud_sumisa",
        "temerosa": "actitud_temorosa",
        "retraida": "actitud_retraida",
        "retraída": "actitud_retraida",
        "apatica": "actitud_apatica",
        "apática": "actitud_apatica",
        "seductora": "actitud_seductora",
        "dependiente": "actitud_dependiente",
        "infantilizada": "actitud_infantilizada",
        "desafectada": "actitud_desafectada",
        "desconfiada": "actitud_desconfiada",
        "ironica": "actitud_ironica",
        "irónica": "actitud_ironica",

        # Nivel de conciencia
        "vigil": "nivel_vigil",
        "somnoliente": "nivel_somnoliente",
        "letargico": "nivel_letargico",
        "letárgico": "nivel_letargico",
        "estupor": "nivel_estupor",
        "obnubilado": "nivel_obnubilado",
        "coma": "nivel_coma",

        # Orientación
        "orientacion en persona": "ori_persona",
        "orientación en persona": "ori_persona",
        "orientacion en tiempo": "ori_tiempo",
        "orientación en tiempo": "ori_tiempo",
        "orientacion en espacio": "ori_espacio",
        "orientación en espacio": "ori_espacio",
        "orientacion en situacion": "ori_situacion",
        "orientación en situación": "ori_situacion",
        "orientacion en contexto social": "ori_contexto",
        "orientación en contexto social": "ori_contexto",

        # Atención
        "euprosexico": "aten_euprosexico",
        "euproséxico": "aten_euprosexico",
        "hiperprosexica": "aten_hiperprosexica",
        "paraporsexico": "aten_paraporsexico",
        "fatigable": "aten_fatigable",

        # Lenguaje
        "eulalico": "len_eulalico",
        "eulálico": "len_eulalico",
        "bradilalia": "len_bradilalia",
        "taquilalia": "len_taquilalia",
        "mutismo": "len_mutismo",

        # Afectividad (solo algunas clave)
        "eutimico": "afecto_eutimico",
        "eutímico": "afecto_eutimico",
        "disforico": "afecto_disforico",
        "disfórico": "afecto_disforico",
        "euporico": "afecto_euporico",
        "eupórico": "afecto_euporico",
        "irritabilidad": "afecto_irritable",
        "expansividad afectiva": "afecto_expansivo",

        # Inteligencia
        "impresiona normal": "intel_normal",
        "impresiona por debajo del promedio": "intel_bajo",
        "impresiona por encima del promedio": "intel_alto",
        "impresiona con retraso": "intel_retraso",

        # Velocidad del pensamiento
        "eupsiquico": "pens_velocidad_eupsiquico",
        "eupsíquico": "pens_velocidad_eupsiquico",
        "bradipsiquico": "pens_velocidad_bradipsiquico",
        "bradipsíquico": "pens_velocidad_bradipsiquico",
        "taquipsiquico": "pens_velocidad_taquipsiquico",
        "taquipsíquico": "pens_velocidad_taquipsiquico",

        # Curso del pensamiento
        "bloqueos del pensamiento": "pens_curso_bloqueos",
        "perseveracion": "pens_curso_perseveracion",
        "perseveración": "pens_curso_perseveracion",
        "fuga de ideas": "pens_curso_fuga",
        "pensamiento empobrecido": "pens_curso_empobrecido",
        "tangencialidad": "pens_curso_tangencialidad",
        "incoherencia / desorganizacion": "pens_curso_incoherencia",
        "incoherencia / desorganización": "pens_curso_incoherencia",
        "circunstancialidad": "pens_curso_circunstancialidad",

        # Tipo / Contenido pensamiento
        "ideas delirantes": "pens_tipo_delirantes",
        "ideas obsesivas": "pens_tipo_obsesivas",
        "normales": "pens_tipo_normales",
        "ideas de culpa": "pens_contenido_culpa",
        "ideas de persecucion / perjuicio": "pens_contenido_persecucion",
        "ideas de persecución / perjuicio": "pens_contenido_persecucion",
        "ideas de autorreproche": "pens_contenido_autorreproche",
        "ideas de referencia": "pens_contenido_referencia",
        "ideas hipocondriacas": "pens_contenido_hipocondria",
        "ideas hipocondríacas": "pens_contenido_hipocondria",
        "ideas de grandeza": "pens_contenido_grandeza",
        "ideas de ruina o pobreza": "pens_contenido_ruina",
        "ideas de nihilismo": "pens_contenido_nihilismo",

        # Psicomotricidad (clave “sin alteraciones”)
        "sin alteraciones": "psico_sin",
        "agitacion psicomotora": "psico_agitacion",
        "agitación psicomotora": "psico_agitacion",
        "lentitud psicomotora": "psico_lentitud",

        # Sensopercepción (muy resumido)
        "alucinaciones auditivas": "senso_aauditivas",
        "alucinaciones visuales": "senso_avisuales",
        "ilusiones visuales": "senso_ilusiones_visuales",
        "despersonalizacion": "senso_despers",
        "despersonalización": "senso_despers",
        "desrealizacion": "senso_desreal",
        "desrealización": "senso_desreal",

        # Juicio de realidad
        "conservado": "juicio_conservado",
        "alterado": "juicio_alterado",

        # Conciencia de enfermedad
        "ausente": "conciencia_ausente",
        "presente": "conciencia_presente",
    }
    return CANON.get(base, base)

def _is_placeholder(note: str) -> bool:
    if note is None:
        return True
    note = _strip_html_tags(note).strip()
    return note in ("", "**")

def _g_word(masc: str, fem: str, genero: str | None) -> str:
    g = (genero or "").strip().lower()
    if g == "femenino":
        return fem
    return masc  # por defecto masculino

def _join_list(items):
    items = [i for i in items if i]
    if not items:
        return ""
    if len(items) == 1:
        return items[0]
    if len(items) == 2:
        return f"{items[0]} y {items[1]}"
    return ", ".join(items[:-1]) + f" y {items[-1]}"

def generar_narrativa_examen_v3(selected_items, genero=None, edad=None) -> str:
    """
    Genera texto clínico fluido, con género/edad opcional, sin HTML,
    respetando tus categorías y notas.
    """
    if not selected_items:
        encabezado = ""
        if genero or edad:
            e = f" de {int(edad)} años" if isinstance(edad, (int, float)) and edad else ""
            encabezado = f"Paciente {_g_word('masculino', 'femenina', genero)}{e}"
            return f"{encabezado}, sin alteraciones significativas en las áreas evaluadas del examen mental."
        return "Paciente sin alteraciones significativas en las áreas evaluadas del examen mental."

    # Normaliza entradas
    norm = []
    for it in selected_items:
        cat = _strip_html_tags(it.get("categoria"))
        sub = _canonize_display(_strip_html_tags(it.get("subcategoria")))
        nota = _strip_html_tags(it.get("notas"))
        norm.append({"cat": cat, "sub": sub, "nota": None if _is_placeholder(nota) else nota})

    # Agrupa por categoría tal como las usas en la UI
    por_cat = {}
    for it in norm:
        por_cat.setdefault(it["cat"], []).append(it)

    frases = []

    # Encabezado
    if genero or edad:
        e = f" de {int(edad)} años" if isinstance(edad, (int, float)) and edad else ""
        frases.append(f"Paciente {_g_word('masculino', 'femenina', genero)}{e}")

    # Aspecto/Vestimenta/Biotipo/Actitud
    # Vestimenta
    if "Vestimenta" in por_cat:
        vest = por_cat["Vestimenta"]
        vest_txt = None
        vest_notas = []
        for v in vest:
            if v["sub"] == "vestimenta_acorde":
                vest_txt = _g_word("vestido acorde a la edad y al contexto", "vestida acorde a la edad y al contexto", genero)
            elif v["sub"] == "vestimenta_no_contexto":
                vest_txt = _g_word("vestido acorde a la edad, no acorde al contexto", "vestida acorde a la edad, no acorde al contexto", genero)
            elif v["sub"] == "vestimenta_desacorde":
                vest_txt = _g_word("vestido desacorde a la edad y al contexto", "vestida desacorde a la edad y al contexto", genero)
            if v["nota"]:
                vest_notas.append(v["nota"])
        if vest_txt:
            if vest_notas:
                vest_txt += f": " + "; ".join(vest_notas)
            frases.append(vest_txt)

    # Biotipo
    if "Biotipo" in por_cat:
        biotipos_map = {
            "biotipo_picnico": "pícnico",
            "biotipo_astenico": "asténico",
            "biotipo_atletico": "atlético",
            "biotipo_displasico": "displásico",
            "biotipo_ectomorfo": "ectomorfo",
            "biotipo_mesomorfo": "mesomorfo",
            "biotipo_endomorfo": "endomorfo",
        }
        bs = []
        for b in por_cat["Biotipo"]:
            bs.append(biotipos_map.get(b["sub"], _strip_html_tags(b["sub"])))
        bs = [b for b in bs if b]
        if bs:
            frases.append("biotipo " + _join_list(bs))

    # Actitud
    if "Actitud" in por_cat:
        acts = []
        notas = []
        for a in por_cat["Actitud"]:
            acts.append(_strip_html_tags(a["cat"] and a["sub"]).replace("actitud_", "").replace("_", " "))
            if a["nota"]:
                notas.append(a["nota"])
        acts = [a for a in acts if a]
        if acts:
            frases.append(_join_list(acts))
        if notas:
            frases.append("; ".join(notas))

    # Nivel de conciencia
    if "Nivel de conciencia" in por_cat:
        nv = por_cat["Nivel de conciencia"]
        lbl = []
        for n in nv:
            if n["sub"] == "nivel_vigil":
                lbl.append("vigil")
            else:
                lbl.append(_strip_html_tags(n["sub"]))
        if lbl:
            frases.append(_join_list(lbl))

    # Orientación: persona/tiempo/espacio/situación/contexto
    if "Orientación" in por_cat:
        ori = por_cat["Orientación"]
        marks = set()
        for o in ori:
            marks.add(o["sub"])
        partes = []
        if "ori_persona" in marks:
            partes.append("persona")
        if "ori_tiempo" in marks:
            partes.append("tiempo")
        if "ori_espacio" in marks:
            partes.append("espacio")
        if "ori_situacion" in marks:
            partes.append("situación")
        if "ori_contexto" in marks:
            partes.append("contexto social")
        if partes:
            frases.append(_g_word("orientado en ", "orientada en ", genero) + _join_list(partes))

    # Atención
    if "Atención" in por_cat:
        at = set(a["sub"] for a in por_cat["Atención"])
        if "aten_euprosexico" in at:
            frases.append("euproséxico")
        elif "aten_hiperprosexica" in at:
            frases.append("hiperproséxico")
        elif "aten_paraporsexico" in at:
            frases.append("paraprosexia")
        if any(a.get("nota") for a in por_cat["Atención"]):
            frases.append("; ".join([a["nota"] for a in por_cat["Atención"] if a.get("nota")]))

    # Memoria (si la usas)
    if "Memoria" in por_cat:
        mems = [m["sub"] for m in por_cat["Memoria"]]
        if mems:
            frases.append("memoria " + _join_list(mems))

    # Afectividad
    if "Afectividad" in por_cat:
        af_map = {
            "afecto_eutimico": "eutímico",
            "afecto_disforico": "disfórico",
            "afecto_euporico": "eufórico",
            "afecto_irritable": "irritable",
            "afecto_expansivo": "expansivo",
        }
        af = []
        for a in por_cat["Afectividad"]:
            af.append(af_map.get(a["sub"], _strip_html_tags(a["sub"])))
        af = [x for x in af if x]
        if af:
            frases.append("afecto " + _join_list(af))

    # Inteligencia
    if "Inteligencia" in por_cat:
        intel_map = {
            "intel_normal": "impresiona dentro del promedio",
            "intel_bajo": "impresiona por debajo del promedio",
            "intel_alto": "impresiona por encima del promedio",
            "intel_retraso": "impresiona con retraso",
        }
        iv = []
        for i in por_cat["Inteligencia"]:
            iv.append(intel_map.get(i["sub"], _strip_html_tags(i["sub"])))
        iv = [x for x in iv if x]
        if iv:
            frases.append("inteligencia " + _join_list(iv))

    # Lenguaje
    if "Lenguaje" in por_cat:
        l_map = {
            "len_eulalico": "eulálico",
            "len_bradilalia": "bradilálico",
            "len_taquilalia": "taquilálico",
            "len_mutismo": "mutismo",
        }
        ls = []
        notas = []
        for l in por_cat["Lenguaje"]:
            ls.append(l_map.get(l["sub"], _strip_html_tags(l["sub"])))
            if l["nota"]:
                notas.append(l["nota"])
        ls = [x for x in ls if x]
        if ls:
            frases.append("lenguaje " + _join_list(ls))
        if notas:
            frases.append("; ".join(notas))

    # Pensamiento: velocidad, curso, tipo, contenido
    if "Velocidad del pensamiento" in por_cat:
        v_map = {
            "pens_velocidad_eupsiquico": "eupsíquico",
            "pens_velocidad_bradipsiquico": "bradipsíquico",
            "pens_velocidad_taquipsiquico": "taquipsíquico",
        }
        vs = []
        notas = []
        for v in por_cat["Velocidad del pensamiento"]:
            vs.append(v_map.get(v["sub"], _strip_html_tags(v["sub"])))
            if v["nota"]:
                notas.append(v["nota"])
        vs = [x for x in vs if x]
        if vs:
            frases.append("pensamiento " + _join_list(vs))
        if notas:
            frases.append("Vb paciente: " + " ".join(notas))

    if "Curso del pensamiento" in por_cat:
        c_map = {
            "pens_curso_bloqueos": "bloqueos del pensamiento",
            "pens_curso_perseveracion": "perseveración",
            "pens_curso_fuga": "fuga de ideas",
            "pens_curso_empobrecido": "pensamiento empobrecido",
            "pens_curso_tangencialidad": "tangencialidad",
            "pens_curso_incoherencia": "incoherencia/desorganización",
            "pens_curso_circunstancialidad": "circunstancialidad",
        }
        cs = []
        notas = []
        for c in por_cat["Curso del pensamiento"]:
            cs.append(c_map.get(c["sub"], _strip_html_tags(c["sub"])))
            if c["nota"]:
                notas.append(c["nota"])
        cs = [x for x in cs if x]
        if cs:
            frases.append("con " + _join_list(cs))
        if notas:
            # Si hay una sola y parece cita, la mantenemos
            frases.append(" ".join(notas))

    if "Tipo del pensamiento" in por_cat:
        ts = [_strip_html_tags(t["sub"]) for t in por_cat["Tipo del pensamiento"]]
        ts = [x for x in ts if x]
        if ts:
            frases.append("con " + _join_list(ts))

    if "Contenido de pensamiento" in por_cat:
        cont = []
        notas = []
        for t in por_cat["Contenido de pensamiento"]:
            cont.append(_strip_html_tags(t["sub"]))
            if t["nota"]:
                notas.append(t["nota"])
        cont = [x for x in cont if x]
        if cont:
            frases.append(_join_list(cont))
        if notas:
            frases.append("Vb paciente: " + " ".join(notas))

    # Psicomotricidad
    if "Psicomotricidad" in por_cat:
        ps = []
        notas = []
        for p in por_cat["Psicomotricidad"]:
            if p["sub"] == "psico_sin":
                ps.append("sin alteraciones")
            else:
                ps.append(_strip_html_tags(p["sub"]))
            if p["nota"]:
                notas.append(p["nota"])
        ps = [x for x in ps if x]
        if ps:
            frases.append(_join_list(ps))
        if notas:
            frases.append("; ".join(notas))

    # Sensopercepción
    if "Sensopercepción" in por_cat:
        ss = [_strip_html_tags(s["sub"]) for s in por_cat["Sensopercepción"]]
        ss = [x for x in ss if x]
        if ss:
            frases.append("sensopercepción " + _join_list(ss))

    # Juicio
    if "Juicio de realidad" in por_cat:
        j = set(t["sub"] for t in por_cat["Juicio de realidad"])
        if "juicio_conservado" in j:
            frases.append("juicio de realidad conservado")
        elif "juicio_alterado" in j:
            frases.append("juicio de realidad alterado")

    # Conciencia
    if "Conciencia de enfermedad" in por_cat:
        c = set(t["sub"] for t in por_cat["Conciencia de enfermedad"])
        if "conciencia_presente" in c:
            frases.append("conciencia de enfermedad presente")
        elif "conciencia_ausente" in c:
            frases.append("conciencia de enfermedad ausente")

    # Unir todo con comas y punto final. Arreglo de espacios y dobles palabras
    texto = ", ".join([f for f in frases if f])
    texto = re.sub(r"\s{2,}", " ", texto).strip()
    # Arreglos de repeticiones tipo "impresiona impresiona"
    texto = re.sub(r"\b(impresiona)\s+\1\b", r"\1", texto)
    if texto and not texto.endswith("."):
        texto += "."
    return texto

#--------final del bloque examen mental
# ====== Asistente de Enfermedad Actual (helpers + diálogo) ======

def generar_narrativa_ea(datos_paciente, intro, sintomas, desarrollo, citas):
    """
    Ensambla la narrativa de la Enfermedad Actual a partir de las piezas proporcionadas.
    Usa placeholders [CITA_1], [CITA_2], ... que luego se reemplazan por "Vb." formateados.
    """
    partes = []

    # 1) Intro
    if intro and intro.strip():
        partes.append(intro.strip())

    # 2) Síntomas
    if sintomas and sintomas.strip():
        if partes:
            partes.append("La sintomatología principal se caracteriza por:")
        partes.append(sintomas.strip())

    # 3) Desarrollo/estado actual
    if desarrollo and desarrollo.strip():
        if partes:
            partes.append("En cuanto al desarrollo y estado actual:")
        partes.append(desarrollo.strip())

    texto = "\n\n".join(partes)

    # 4) Reemplazar placeholders de citas
    if citas:
        for i, cita in enumerate(citas):
            placeholder = f"[CITA_{i+1}]"
            fuente = (cita.get('fuente') or '').strip()
            quien = (cita.get('quien') or '').strip()
            texto_cita = (cita.get('texto') or '').strip()

            if not texto_cita:
                continue

            if fuente in ("Familiar", "Otro") and quien:
                cita_fmt = f'(Vb. {fuente.lower()} - {quien}: “{texto_cita}”)'
            else:
                fuente_lbl = fuente.lower() if fuente else "paciente"
                cita_fmt = f'(Vb. {fuente_lbl}: “{texto_cita}”)'

            texto = texto.replace(placeholder, cita_fmt)

    return texto


@st.dialog("✍️ Asistente de Redacción: Enfermedad Actual", width="large")
def enfermedad_actual_assistant_dialog():
    """
    Ventana modal para guiar la redacción de la Enfermedad Actual.
    Consistente con el patrón del Examen Mental (vista previa + copiar + usar).
    """
    # Estado del asistente
    if 'ea_asistente_citas' not in st.session_state:
        st.session_state.ea_asistente_citas = []
    if 'ea_narrativa_generada' not in st.session_state:
        st.session_state.ea_narrativa_generada = ""

    st.caption("Rellena las secciones y (opcionalmente) usa [CITA_1], [CITA_2], ... donde quieras insertar verbatims.")
    st.divider()

    # --- 1) Intro sugerida con datos de identificación ---
    st.subheader("1. Inicio y desencadenante")
    id_data = st.session_state.get('datos_identificacion', {})
    genero_str = "femenina" if (id_data.get('genero') or "").lower() == 'femenino' else "masculino"
    edad_val = id_data.get('edad', '')
    ciudad_nac = id_data.get('ciudad_nacimiento', '')
    ciudad_proc = id_data.get('ciudad_procedencia', '')

    sugerencia_intro = (
        f"Paciente {genero_str}"
        f"{f' de {int(edad_val)} años' if str(edad_val).isdigit() else ''}, "
        f"natural de {ciudad_nac or 'N/A'} y procedente de {ciudad_proc or 'N/A'}. "
        "Consulta por..."
    )

    st.text_area(
        "Redacta el párrafo introductorio (inicio del cuadro y desencadenante)",
        value=sugerencia_intro,
        key="ea_asistente_intro",
        height=140
    )

    # --- 2) Sintomatología ---
    st.subheader("2. Sintomatología")
    st.multiselect(
        "Categorías guía (opcional)",
        options=[
            'Síntomas Afectivos', 'Síntomas Psicóticos', 'Síntomas Ansiosos',
            'Alteraciones del Sueño', 'Alteraciones de la Conducta',
            'Síntomas Somáticos', 'Síntomas Cognitivos'
        ],
        key="ea_asistente_cat_sintomas"
    )
    st.text_area(
        "Describe los síntomas. Usa [CITA_1], [CITA_2], ... para insertar verbatims donde correspondan.",
        key="ea_asistente_sintomas",
        height=180
    )

    # --- 3) Citas / Verbatims ---
    st.subheader("3. Citas textuales (verbatims)")
    with st.form("form_add_cita_ea", clear_on_submit=True):
        c1, c2 = st.columns(2)
        fuente = c1.selectbox("Fuente", ["Paciente", "Familiar", "Otro"], key="ea_cita_fuente")
        quien = c2.text_input("¿Quién? (Ej: hija, esposo)", key="ea_cita_quien")
        texto_cita = st.text_area("Texto de la cita", key="ea_cita_texto")
        if st.form_submit_button("Añadir Cita"):
            if (texto_cita or "").strip():
                st.session_state.ea_asistente_citas.append({
                    "fuente": fuente,
                    "quien": quien,
                    "texto": texto_cita
                })
                st.rerun()

    if st.session_state.ea_asistente_citas:
        st.write("**Citas añadidas (usa sus placeholders):**")
        for i, cita in enumerate(st.session_state.ea_asistente_citas):
            placeholder = f"[CITA_{i+1}]"
            quien_lbl = f" ({cita['quien']})" if cita.get('quien') else ""
            st.info(f"**{placeholder}** — {cita['fuente']}{quien_lbl}: “{cita['texto']}”")

    # --- 4) Desarrollo / Estado actual ---
    st.subheader("4. Desarrollo y estado actual")
    st.text_area(
        "Describe la progresión del cuadro y el motivo de consulta actual.",
        key="ea_asistente_desarrollo",
        height=140
    )

    st.divider()

    # --- Generar y mostrar resultado ---
    if st.button("✨ Generar narrativa", type="primary", use_container_width=True):
        st.session_state.ea_narrativa_generada = generar_narrativa_ea(
            datos_paciente=st.session_state.get('datos_identificacion', {}),
            intro=st.session_state.get("ea_asistente_intro", ""),
            sintomas=st.session_state.get("ea_asistente_sintomas", ""),
            desarrollo=st.session_state.get("ea_asistente_desarrollo", ""),
            citas=st.session_state.get("ea_asistente_citas", []),
        )

    if st.session_state.ea_narrativa_generada:
        st.subheader("Vista previa (texto)")
        st.text(st.session_state.ea_narrativa_generada)

        st.markdown("---")
        st.markdown("**Copiar este contenido (botón copiar arriba a la derecha):**")
        st.code(st.session_state.ea_narrativa_generada, language="markdown")

        c1, c2 = st.columns(2)
        if c1.button("✅ Usar en Enfermedad Actual", use_container_width=True):
            st.session_state.ea_temp = st.session_state.ea_narrativa_generada
            st.session_state.ea_asistente_citas = []
            st.session_state.ea_narrativa_generada = ""
            st.rerun()
        if c2.button("Cerrar", use_container_width=True):
            st.rerun()

#---final de helper enfermedad actual------

def upsert_data(table_name, data_dict):
    primary_key_map = {'sesiones': 'id_sesion', 'agenda': 'id_cita'}
    primary_key = primary_key_map.get(table_name, 'id_paciente')
    if not client or not data_dict.get(primary_key): return False

    temp_table_id = None
    try:
        table_id = f"{PROJECT_ID}.{DATASET_ID}.{table_name}"
        table = table_objects[table_name]
        schema_cols = [field.name for field in table.schema]

        cleaned_data = {k: v for k, v in data_dict.items() if k in schema_cols}

        df = pd.DataFrame([cleaned_data])

        for field in table.schema:
            col_name = field.name
            if col_name in df.columns:
                if field.field_type == 'TIME':
                    df[col_name] = df[col_name].apply(
                        lambda t: t.strftime('%H:%M:%S') if pd.notna(t) and isinstance(t, datetime.time) else None
                    )
                elif field.field_type == 'DATE':
                     df[col_name] = pd.to_datetime(df[col_name], errors='coerce').dt.date

        job_config = bigquery.LoadJobConfig(schema=table.schema)
        temp_table_id = f"{PROJECT_ID}.{DATASET_ID}.temp_{table_name}_{generate_unique_id()}"
        load_job = client.load_table_from_dataframe(df, temp_table_id, job_config=job_config)
        load_job.result()

        update_cols = [f"T.`{col}` = S.`{col}`" for col in df.columns if col != primary_key]
        insert_cols = [f"`{col}`" for col in df.columns]
        source_cols = [f"S.`{col}`" for col in df.columns]

        merge_sql = f"""MERGE `{table_id}` T USING `{temp_table_id}` S ON T.`{primary_key}` = S.`{primary_key}`
                            WHEN MATCHED THEN UPDATE SET {', '.join(update_cols)}
                            WHEN NOT MATCHED THEN INSERT ({', '.join(insert_cols)}) VALUES ({', '.join(source_cols)})"""

        if not update_cols:
            merge_sql = f"""MERGE `{table_id}` T USING `{temp_table_id}` S ON T.`{primary_key}` = S.`{primary_key}`
                                WHEN NOT MATCHED THEN INSERT ({', '.join(insert_cols)}) VALUES ({', '.join(source_cols)})"""

        client.query(merge_sql).result()
        return True
    except Exception as e:
        st.error(f"Error al guardar en '{table_name}': {e}"); return False
    finally:
        if temp_table_id: client.delete_table(temp_table_id, not_found_ok=True)

def delete_patient(patient_id):
    if not client or not patient_id: return False
    tables_to_delete_from = [
        'agenda', 'sesiones', 'comprension_psicodinamica', 'tratamientos',
        'diagnosticos', 'examen_mental', 'habitos_psicobiologicos',
        'antecedentes_familiares', 'antecedentes_personales', 'enfermedad_actual',
        'motivo_de_consulta', 'datos_administrativos', 'datos_identificacion',
        'pacientes'
    ]
    try:
        for table_name in tables_to_delete_from:
            query = f"DELETE FROM `{PROJECT_ID}.{DATASET_ID}.{table_name}` WHERE id_paciente = @patient_id"
            job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ScalarQueryParameter("patient_id", "STRING", patient_id)])
            client.query(query, job_config=job_config).result()
        st.cache_data.clear()
        st.success("Paciente y todos sus datos han sido eliminados permanentemente.")
        return True
    except Exception as e:
        st.error(f"Error al eliminar el paciente: {e}"); return False

def delete_record(table_name, record_id, id_column):
    if not client or not record_id: return False
    try:
        query = f"DELETE FROM `{PROJECT_ID}.{DATASET_ID}.{table_name}` WHERE {id_column} = @record_id"
        job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ScalarQueryParameter("record_id", "STRING", record_id)])
        client.query(query, job_config=job_config).result()
        st.success(f"Registro eliminado de la tabla '{table_name}'.")
        return True
    except Exception as e:
        st.error(f"Error al eliminar el registro: {e}"); return False

@st.cache_data(ttl=300)
def get_patient_data_from_db(_client, _table_objects_dict, patient_id):
    if not _client or not patient_id or not _table_objects_dict: return {}
    data = {}
    table_schemas = {name: [field.name for field in table.schema] for name, table in _table_objects_dict.items()}
    tables_to_join = [
        'datos_identificacion', 'datos_administrativos', 'motivo_de_consulta','enfermedad_actual',
        'antecedentes_personales', 'antecedentes_familiares', 'habitos_psicobiologicos',
        'examen_mental', 'diagnosticos', 'tratamientos', 'comprension_psicodinamica'
    ]
    joins, select_aliases = "", []
    for table_name in tables_to_join:
        alias = ''.join([part[0] for part in table_name.split('_')])
        joins += f"LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.{table_name}` AS {alias} ON p.id_paciente = {alias}.id_paciente \n"
        select_aliases.append(f"{alias}.*")
    main_query = f"SELECT p.*, {', '.join(select_aliases)} FROM `{PROJECT_ID}.{DATASET_ID}.pacientes` AS p {joins} WHERE p.id_paciente = @patient_id LIMIT 1"
    job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ScalarQueryParameter("patient_id", "STRING", patient_id)])
    try:
        df_main = _client.query(main_query, job_config=job_config).to_dataframe()
        all_tables = ['pacientes'] + tables_to_join
        for table in all_tables: data[table] = {}
        if not df_main.empty:
            main_data_row = df_main.to_dict('records')[0]
            for col, value in main_data_row.items():
                if isinstance(value, (list, np.ndarray)) or pd.notna(value):
                    for tbl in all_tables:
                        if col in table_schemas.get(tbl, []):
                            data[tbl][col] = value
                            break
    except Exception as e:
        st.error(f"Error al cargar datos principales del paciente: {e}"); return {}
    for table_name in ['sesiones', 'agenda']:
        try:
            query = f"SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.{table_name}` WHERE id_paciente = @patient_id"
            job_config_many = bigquery.QueryJobConfig(query_parameters=[bigquery.ScalarQueryParameter("patient_id", "STRING", patient_id)])
            df = _client.query(query, job_config_many).to_dataframe()
            data[table_name] = df.to_dict('records') if not df.empty else []
        except: data[table_name] = []
    return data

def load_data_for_current_patient():
    patient_id = st.session_state.current_patient_id
    with st.spinner("Cargando datos del paciente..."):
        data = get_patient_data_from_db(client, table_objects, patient_id)
        for table, record in data.items():
            if table not in ['sesiones', 'agenda']: st.session_state[table] = record or {}

        def robust_list_loader(source_dict, key):
            value = source_dict.get(key)
            if isinstance(value, np.ndarray):
                return value.tolist()
            elif value is not None:
                return list(value)
            return []

        st.session_state.hijos_list = robust_list_loader(st.session_state.get('datos_identificacion', {}), 'hijos')
        st.session_state.antecedentes_medicos_list = robust_list_loader(st.session_state.get('antecedentes_personales', {}), 'antecedentes_medicos')
        st.session_state.antecedentes_psiquiatricos_list = robust_list_loader(st.session_state.get('antecedentes_personales', {}), 'antecedentes_psiquiatricos')
        st.session_state.familiares_list = robust_list_loader(st.session_state.get('antecedentes_familiares', {}), 'familiares')
        st.session_state.selected_msea_items = robust_list_loader(st.session_state.get('examen_mental', {}), 'msea_items')
        st.session_state.current_patient_diagnoses = robust_list_loader(st.session_state.get('diagnosticos', {}), 'diagnosticos')
        st.session_state.tratamientos_farmacologicos_list = robust_list_loader(st.session_state.get('tratamientos', {}), 'farmacologico')
        st.session_state.tratamientos_psiquiatricos_list = robust_list_loader(st.session_state.get('tratamientos', {}), 'psiquiatrico')

        st.session_state.sesiones_list = data.get('sesiones', [])
        st.session_state.agenda_list = data.get('agenda', [])
    st.session_state.loaded_patient_id = patient_id

def on_select_patient(patient_id):
    st.session_state.current_patient_id = patient_id
    st.session_state.page = 'ficha'
    st.rerun()

@st.cache_data(ttl=3600)
def load_reference_data(_client):
    if not _client: return {}, [], [], {}, [], {}
    try:
        df_paises = client.query(f"SELECT country_name, country_id FROM `{PROJECT_ID}.{DATASET_ID}.Paises`").to_dataframe()
        country_options = [''] + sorted(df_paises['country_name'].unique().tolist())
        cities_by_country = {country: [''] + sorted(df_paises[df_paises['country_name'] == country]['country_id'].tolist()) for country in df_paises['country_name'].unique()}
        df_em = client.query(f"SELECT Categoria, `Sub- categoria` FROM `{PROJECT_ID}.{DATASET_ID}.Examen_mental`").to_dataframe()
        df_em.columns = ['Categoria', 'Sub_categoria']
        msea_data = {cat: sorted(group['Sub_categoria'].tolist()) for cat, group in df_em.groupby('Categoria')}
        msea_order = ['Aspecto','Vestimenta','Biotipo','Actitud','Nivel de conciencia','Estado de conciencia','Orientación','Atención','Memoria','Lenguaje','Afectividad','Inteligencia','Velocidad del pensamiento','Curso del pensamiento','Tipo del pensamiento','Contenido de pensamiento','Sensopercepción','Juicio de realidad','Psicomotricidad','Conciencia de enfermedad']
        df_diag = client.query(f"SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.Diagnostico`").to_dataframe()
        diag_categories = sorted(list(set(df_diag['Categoría'].dropna())))
        grouped_diag_data = df_diag.groupby(['Categoría', 'Subcategoría']).apply(lambda x: x.to_dict('records')).to_dict()
        return msea_data, msea_order, diag_categories, grouped_diag_data, country_options, cities_by_country
    except Exception as e:
        st.error(f"Error al cargar datos de referencia: {e}"); return {}, [], [], {}, [], {}

# --- NAVEGACIÓN LATERAL ---
with st.sidebar:
    st.markdown("<div class='sidebar-title'>Áreas</div>", unsafe_allow_html=True)
    st.markdown("---")
    if st.button("👤 Mi Consultorio", use_container_width=True, type="primary" if st.session_state.page == 'consultorio' else "secondary"):
        st.session_state.page = 'consultorio'
        st.session_state.current_patient_id = None
        st.session_state.loaded_patient_id = None
        st.rerun()
    if st.button("🗓️ Agenda", use_container_width=True, type="primary" if st.session_state.page == 'agenda' else "secondary"):
        st.session_state.page = 'agenda'; st.rerun()
    if st.button("📊 Resumen Administrativo", use_container_width=True, type="primary" if st.session_state.page == 'resumen' else "secondary"):
        st.session_state.page = 'resumen'; st.rerun()

# --- RENDERIZADO DE PÁGINAS ---
if not st.session_state.welcome_seen:
    st.title("Bienvenido a MiEspacioTerapeutico")
    st.subheader("Tu centro de gestión clínica digital.")
    st.markdown("---")
    st.info("Organiza la información de tus pacientes, gestiona tus sesiones y lleva un control administrativo de tu consulta de forma sencilla y segura.")
    if st.button("Empezar a Gestionar", type="primary"):
        st.session_state.welcome_seen = True; st.rerun()

elif st.session_state.page == 'consultorio':
    st.markdown('<h1 class="page-title">Mi Consultorio</h1>', unsafe_allow_html=True)

    @st.cache_data(ttl=60)
    def get_all_patients_with_status(_client):
        if not _client: return []
        query = f"""SELECT p.id_paciente, p.nombre_completo, d.estado_paciente FROM `{PROJECT_ID}.{DATASET_ID}.pacientes` AS p LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.datos_administrativos` AS d ON p.id_paciente = d.id_paciente ORDER BY p.nombre_completo"""
        try: return _client.query(query).to_dataframe().to_dict('records')
        except: return []

    patients = get_all_patients_with_status(client)

    col1, col2 = st.columns([2, 1], vertical_alignment="bottom")
    with col1:
        st.subheader("Buscar Paciente")
        search_query = st.text_input("Buscar Paciente", placeholder="Escribe un nombre para buscar...", label_visibility="collapsed")
    with col2:
        if st.button("✚ Crear Nuevo Paciente", type="primary", use_container_width=True):
            @st.dialog("Crear Nuevo Paciente")
            def create_patient_dialog():
                with st.form("form_create_patient_dialog"):
                    name = st.text_input("Nombre y Apellido del Paciente")
                    if st.form_submit_button("Crear y Abrir Ficha", type="primary"):
                        if name and name.strip():
                            patient_id = generate_unique_id()
                            if upsert_data('pacientes', {"id_paciente": patient_id, "nombre_completo": name}):
                                st.success(f"Paciente '{name}' creado."); on_select_patient(patient_id)
                            else: st.error("No se pudo crear el paciente.")
            create_patient_dialog()

    st.divider()

    filtered_patients = patients
    if search_query:
        filtered_patients = [p for p in patients if search_query.lower() in p.get('nombre_completo', '').lower()]

    active_patients = [p for p in filtered_patients if p.get('estado_paciente') == 'Activo']
    inactive_patients = [p for p in filtered_patients if p.get('estado_paciente') != 'Activo']

    st.subheader("Mis Pacientes")
    tab_todos, tab_activos, tab_inactivos = st.tabs([
        f"👥 Todos ({len(filtered_patients)})",
        f"✅ Activos ({len(active_patients)})",
        f"🚫 Inactivos ({len(inactive_patients)})"
    ])

    def display_patient_list(patient_list, key_prefix):
        if not patient_list:
            st.info("No se encontraron pacientes que coincidan con los filtros seleccionados.")
            return

        @st.dialog("Confirmar Eliminación")
        def delete_patient_dialog(patient_id, patient_name):
            st.warning(f"¿Estás seguro de que quieres eliminar a **{patient_name}**?")
            st.markdown("Esta acción es **irreversible** y borrará toda la historia clínica, sesiones y datos administrativos asociados.")
            c1, c2 = st.columns(2)
            if c1.button("Cancelar", use_container_width=True): st.rerun()
            if c2.button("Eliminar Permanentemente", type="primary", use_container_width=True):
                if delete_patient(patient_id):
                    st.session_state.page = 'consultorio'
                    st.session_state.current_patient_id = None
                    st.rerun()

        header_cols = st.columns([4, 2, 1, 1])
        header_cols[0].markdown('<p class="column-header">Nombre Completo</p>', unsafe_allow_html=True)
        header_cols[1].markdown('<p class="column-header">Estado</p>', unsafe_allow_html=True)

        st.divider()

        for patient in patient_list:
            data_cols = st.columns([4, 2, 1, 1], vertical_alignment="center")
            data_cols[0].markdown(f"**{patient['nombre_completo']}**")

            status = patient.get('estado_paciente') or 'Indefinido'
            color = "green" if status == "Activo" else "orange"
            data_cols[1].markdown(f"Estado: :{color}[{status}]")

            with data_cols[2]:
                st.button("Abrir Ficha", key=f"{key_prefix}_view_{patient['id_paciente']}", on_click=on_select_patient, args=(patient['id_paciente'],), use_container_width=True)
            with data_cols[3]:
                if st.button("Eliminar", key=f"{key_prefix}_del_{patient['id_paciente']}", type="secondary", use_container_width=True):
                    delete_patient_dialog(patient['id_paciente'], patient['nombre_completo'])
            st.divider()

    with tab_todos:
        display_patient_list(filtered_patients, key_prefix="all")
    with tab_activos:
        display_patient_list(active_patients, key_prefix="active")
    with tab_inactivos:
        display_patient_list(inactive_patients, key_prefix="inactive")

elif st.session_state.page == 'ficha' and st.session_state.current_patient_id:
    if st.session_state.current_patient_id != st.session_state.get('loaded_patient_id'):
        load_data_for_current_patient()

    patient_id = st.session_state.current_patient_id
    patient_name = st.session_state.get('pacientes', {}).get('nombre_completo', '')

    st.markdown(f'<p class="patient-name-header">{patient_name}</p>', unsafe_allow_html=True)
    st.divider()

    tab1, tab2, tab_sesiones, tab5, tab3, tab4 = st.tabs([
        "📝 Ficha Clínica", "🗂️ Datos Administrativos", "🗣️ Sesiones",
        "🗓️ Agenda del Paciente", "🧠 Comprensión Psicodinámica", "📄 Resumen"
    ])

    with tab1:
        min_date_allowed = datetime.date(1940, 1, 1)
        msea_data, msea_order, diag_categories, grouped_diag_data, country_options, cities_by_country = load_reference_data(client)
        with st.expander("1. Datos de Identificación", expanded=True):
            data = st.session_state.get('datos_identificacion', {})
            c1, c2 = st.columns(2)
            with c1:
                st.text_input("Nombre Completo", value=patient_name, disabled=True, key=f"di_name_{patient_id}")
                g_opts=['', 'Masculino', 'Femenino']; st.selectbox("Género", g_opts, key="di_genero", index=g_opts.index(data.get('genero')) if data.get('genero') in g_opts else 0)
                pais_nacimiento = st.selectbox("País de Nacimiento", country_options, key="di_pais_nacimiento", index=country_options.index(data.get('pais_nacimiento')) if data.get('pais_nacimiento') in country_options else 0)
                ciudades_nacimiento = cities_by_country.get(pais_nacimiento, [''])
                st.selectbox("Ciudad de Nacimiento", ciudades_nacimiento, key="di_ciudad_nacimiento", index=ciudades_nacimiento.index(data.get('ciudad_nacimiento')) if data.get('ciudad_nacimiento') in ciudades_nacimiento else 0)
                c_opts = ['','Soltero/a', 'Casado/a', 'Divorciado/a', 'Viudo/a', 'Unión libre']; st.selectbox("Estado Civil", c_opts, key="di_estado_civil", index=c_opts.index(data.get('estado_civil')) if data.get('estado_civil') in c_opts else 0)
                if st.session_state.get('di_estado_civil') not in ['', 'Soltero/a']:
                    st.date_input("Desde cuándo (Estado Civil)", key="di_desde_cuando_estado_civil", value=safe_date_converter(data.get('desde_cuando_estado_civil')), min_value=min_date_allowed)
                l_opts = ['', 'Empleado', 'Desempleado', 'Estudiante', 'Jubilado', 'Nunca ha trabajado']; st.selectbox("Estado Laboral", l_opts, key="di_estado_laboral", index=l_opts.index(data.get('estado_laboral')) if data.get('estado_laboral') in l_opts else 0)
                if st.session_state.get('di_estado_laboral') in ['Empleado', 'Desempleado', 'Estudiante']:
                    st.text_input("Ocupación/Posición", key="di_posicion_ocupacion", value=data.get('posicion_ocupacion', ''))
                    st.date_input("Desde cuándo (Laboral)", key="di_desde_cuando_laboral", value=safe_date_converter(data.get('desde_cuando_laboral')), min_value=min_date_allowed)
            with c2:
                st.number_input("Edad", min_value=0, step=1, key="di_edad", value=int(data.get('edad', 0)))
                r_opts = ['', 'Catolicismo', 'Cristianismo (Otras ramas)', 'Judaísmo', 'Islam', 'Budismo', 'Hinduismo', 'Agnóstico', 'Ateo', 'Otra']; st.selectbox("Religión", r_opts, key="di_religion", index=r_opts.index(data.get('religion')) if data.get('religion') in r_opts else 0)
                if st.session_state.get('di_religion') not in ['', 'Agnóstico', 'Ateo']:
                    p_opts = ['', 'Practicante', 'No practicante', 'Ocasional']; st.selectbox("Nivel de práctica religiosa", p_opts, key="di_nivel_de_practica_religiosa", index=p_opts.index(data.get('nivel_de_practica_religiosa')) if data.get('nivel_de_practica_religiosa') in p_opts else 0)
                pais_procedencia = st.selectbox("País de Procedencia", country_options, key="di_pais_procedencia", index=country_options.index(data.get('pais_procedencia')) if data.get('pais_procedencia') in country_options else 0)
                ciudades_procedencia = cities_by_country.get(pais_procedencia, [''])
                st.selectbox("Ciudad de Procedencia", ciudades_procedencia, key="di_ciudad_procedencia", index=ciudades_procedencia.index(data.get('ciudad_procedencia')) if data.get('ciudad_procedencia') in ciudades_procedencia else 0)
                if st.session_state.get('di_pais_procedencia') != st.session_state.get('di_pais_nacimiento') or st.session_state.get('di_ciudad_procedencia') != st.session_state.get('di_ciudad_nacimiento'):
                    st.date_input("Desde cuándo (Procedencia)", key="di_desde_cuando_procedencia", value=safe_date_converter(data.get('desde_cuando_procedencia')), min_value=min_date_allowed)
                e_opts = ['', 'Sin estudios', 'Primaria', 'Secundaria', 'Universitaria', 'Máster', 'Doctorado']; st.selectbox("Nivel Educativo", e_opts, key="di_nivel_educativo", index=e_opts.index(data.get('nivel_educativo')) if data.get('nivel_educativo') in e_opts else 0)
                if st.session_state.get('di_nivel_educativo') in ['Universitaria', 'Máster', 'Doctorado']:
                    st.text_input("Área de estudio", key="di_area_de_estudio", value=data.get('area_de_estudio', ''))
                st.number_input("Número de Parejas Anteriores", min_value=0, step=1, key="di_numero_de_parejas_anteriores", value=int(data.get('numero_de_parejas_anteriores', 0)))
            st.markdown("---")
            tiene_hijos_actual = "Sí" if st.session_state.hijos_list else "No"
            if st.radio("¿Tiene hijos?", ["No", "Sí"], key="di_tiene_hijos", horizontal=True, index=["No", "Sí"].index(tiene_hijos_actual)) == "Sí":
                with st.form("form_add_hijo", clear_on_submit=True):
                    c1,c2,c3 = st.columns([2,1,1])
                    nombre_hijo = c1.text_input("Nombre del Hijo/a")
                    edad_hijo = c2.number_input("Edad", min_value=0, step=1)
                    if c3.form_submit_button("Añadir"):
                        if nombre_hijo: st.session_state.hijos_list.append({"nombre": nombre_hijo, "edad": int(edad_hijo)}); st.rerun()
                if st.session_state.hijos_list:
                    st.write("**Hijos Registrados:**")
                    for i, hijo in enumerate(list(st.session_state.hijos_list)):
                        c1, c2 = st.columns([4, 1]); c1.markdown(f"- {hijo['nombre']} ({hijo['edad']} años)")
                        if c2.button("X", key=f"del_hijo_{i}"): st.session_state.hijos_list.pop(i); st.rerun()
            elif st.session_state.hijos_list: st.session_state.hijos_list = []; st.rerun()
            if st.button("💾 Guardar Datos de Identificación", key="save_identificacion", use_container_width=True):
                with st.spinner("Guardando..."):
                    ident_data = {'id_paciente': patient_id, 'nombre_completo': patient_name, 'edad': st.session_state.di_edad, 'genero': st.session_state.di_genero, 'pais_nacimiento': st.session_state.di_pais_nacimiento, 'ciudad_nacimiento': st.session_state.di_ciudad_nacimiento, 'estado_civil': st.session_state.di_estado_civil, 'desde_cuando_estado_civil': safe_date_converter(st.session_state.get('di_desde_cuando_estado_civil')), 'estado_laboral': st.session_state.di_estado_laboral, 'posicion_ocupacion': st.session_state.get('di_posicion_ocupacion'), 'desde_cuando_laboral': safe_date_converter(st.session_state.get('di_desde_cuando_laboral')), 'religion': st.session_state.di_religion, 'nivel_de_practica_religiosa': st.session_state.get('di_nivel_de_practica_religiosa'), 'pais_procedencia': st.session_state.di_pais_procedencia, 'ciudad_procedencia': st.session_state.di_ciudad_procedencia, 'desde_cuando_procedencia': safe_date_converter(st.session_state.get('di_desde_cuando_procedencia')), 'nivel_educativo': st.session_state.di_nivel_educativo, 'area_de_estudio': st.session_state.get('di_area_de_estudio'), 'numero_de_parejas_anteriores': st.session_state.di_numero_de_parejas_anteriores, 'hijos': st.session_state.hijos_list}
                    if upsert_data('datos_identificacion', ident_data):
                        st.success("✅ Datos de identificación guardados.")
                        st.cache_data.clear()
                        st.session_state.loaded_patient_id = None
                        time.sleep(1); st.rerun()

                        # ======================= MOTIVO + ENFERMEDAD ACTUAL =======================

        with st.expander("2. Motivo de Consulta y Enfermedad Actual", expanded=True):

            # --- Motivo de consulta ---
            st.write("Motivo de Consulta")
            motivo_content = st_quill(
                value=st.session_state.get('motivo_de_consulta', {}).get('motivo_de_consulta', ''),
                key="mc_motivo_quill"
            )

            st.markdown("---")

            # --- Enfermedad Actual (con asistente) ---
            st.write("Enfermedad Actual")

            # Buffer local para el editor (como en Examen Mental con narrativa_temp)
            if "ea_temp" not in st.session_state:
                st.session_state.ea_temp = st.session_state.get('enfermedad_actual', {}).get('enfermedad_actual', '')

            # Botón para abrir el asistente (modal)
            if st.button("✍️ Asistente de Redacción de Enfermedad Actual", use_container_width=True):
                # Inicializamos/limpiamos estado del asistente
                st.session_state.ea_asistente_citas = []
                st.session_state.ea_narrativa_generada = ""
                enfermedad_actual_assistant_dialog()

            # Editor enriquecido con el buffer
            enfermedad_content = st_quill(
                value=st.session_state.ea_temp,
                key="ea_enfermedad_quill"
            )

            st.caption("Tip: puedes generar el texto con el asistente y luego ajustar manualmente aquí.")

            # Guardar en base de datos motivo + enfermedad actual
            if st.button("💾 Guardar Motivo y Enfermedad", key="save_motivo", use_container_width=True):
                with st.spinner("Guardando..."):
                    # Actualizamos el buffer con lo que hay en el editor
                    st.session_state.ea_temp = enfermedad_content

                    flag1 = upsert_data('motivo_de_consulta', {
                        'id_paciente': patient_id,
                        'motivo_de_consulta': motivo_content
                    })
                    flag2 = upsert_data('enfermedad_actual', {
                        'id_paciente': patient_id,
                        'enfermedad_actual': st.session_state.ea_temp
                    })
                    if flag1 and flag2:
                        st.success("✅ Motivo y Enfermedad Actual guardados.")
                        st.cache_data.clear()
                        st.session_state.loaded_patient_id = None
                        time.sleep(1)
                        st.rerun()

        # ======================= EXAMEN MENTAL (NUEVO) =======================
        with st.expander("3. Examen Mental"):
            c1, c2 = st.columns(2)
            c1.selectbox("Categoría", [''] + msea_order, key="em_cat_sel")
            sub_opts = msea_data.get(st.session_state.em_cat_sel, [])
            c2.selectbox("Subcategoría", [''] + sub_opts, key="em_subcat_sel", disabled=not st.session_state.em_cat_sel)

            # --- Form para añadir ítems ---
            with st.form("em_form", clear_on_submit=True):
                st.text_area("Notas Adicionales", key="em_notas")
                add_disabled = not (st.session_state.em_cat_sel and st.session_state.em_subcat_sel)
                if st.form_submit_button("Añadir Ítem", disabled=add_disabled):
                    st.session_state.selected_msea_items.append(
                        {
                            'categoria': st.session_state.em_cat_sel,
                            'subcategoria': st.session_state.em_subcat_sel,
                            'notas': st.session_state.em_notas
                        }
                    )
                    st.rerun()

            # --- Ítems Registrados en un expander (plegado para ahorrar espacio) ---
            total_items = len(st.session_state.selected_msea_items or [])
            with st.expander(f"Ítems registrados ({total_items})", expanded=False):
                if st.session_state.selected_msea_items:
                    for i, item in enumerate(list(st.session_state.selected_msea_items)):
                        c1, c2 = st.columns([5, 1])
                        c1.markdown(f"- **{item['categoria']} / {item['subcategoria']}**: *{item.get('notas','')}*")
                        if c2.button("Eliminar", key=f"del_msea_{i}"):
                            st.session_state.selected_msea_items.pop(i)
                            st.rerun()
                else:
                    st.caption("Aún no hay ítems registrados.")

            st.markdown("---")
            st.write("Narrativa del Examen / Observaciones Generales:")

            # Estado inicial del editor
            if "narrativa_temp" not in st.session_state:
                st.session_state.narrativa_temp = st.session_state.get('examen_mental', {}).get('observaciones_generales', '')

            # Estado para el modal de narrativa generada
            if "narrativa_generada" not in st.session_state:
                st.session_state.narrativa_generada = ""

            # Editor enriquecido (para pegar manualmente si quieres)
            obs_generales_content = st_quill(
                value=st.session_state.narrativa_temp,
                key="em_observaciones_generales_quill"
            )

            # Modal de vista previa + copiar (sin "Usar en Examen Mental")
            @st.dialog("Narrativa generada", width="large")
            def show_narrativa_dialog():
                texto = st.session_state.get("narrativa_generada") or "No se generó contenido."
                st.markdown("**Vista previa (texto plano):**")
                st.text(texto)

                st.markdown("---")
                st.markdown("**Copiar este contenido (usa el botón copiar arriba a la derecha):**")
                st.code(texto, language="markdown")

                if st.button("Cerrar", use_container_width=True):
                    st.rerun()

            # Generar narrativa y abrir modal (usa SIEMPRE la V3 en texto plano)
            def callback_generar_narrativa():
                di = st.session_state.get('datos_identificacion', {})
                genero = di.get('genero')
                edad = di.get('edad')

                texto = generar_narrativa_examen_v3(
                    st.session_state.selected_msea_items,
                    genero=genero,
                    edad=edad
                )

                st.session_state.narrativa_generada = texto
                show_narrativa_dialog()

            st.button("✨ Generar Narrativa Automática", key="gen_examen_mental", on_click=callback_generar_narrativa)

            # Guardar en base de datos lo que hay actualmente en el editor
            if st.button("💾 Guardar Examen Mental", key="save_examen_mental", use_container_width=True):
                st.session_state.narrativa_temp = obs_generales_content
                with st.spinner("Guardando..."):
                    examen_mental_data = {
                        'id_paciente': patient_id,
                        'msea_items': st.session_state.selected_msea_items,
                        'observaciones_generales': obs_generales_content
                    }
                    if upsert_data('examen_mental', examen_mental_data):
                        st.success("✅ Examen Mental guardado.")
                        st.cache_data.clear()
                        st.session_state.loaded_patient_id = None
                        time.sleep(1)
                        st.rerun()

        # ==================== FIN EXAMEN MENTAL (NUEVO) ======================

        with st.expander("4. Antecedentes Personales"):
            st.subheader("Antecedentes Médicos")
            with st.form("form_ap_medico", clear_on_submit=True):
                c1,c2=st.columns(2); cond=c1.text_input("Condición"); desde=c2.date_input("Fecha Diag.", value=None, min_value=min_date_allowed); notas=st.text_area("Notas")
                if st.form_submit_button("Añadir Ant. Médico"):
                    if cond: st.session_state.antecedentes_medicos_list.append({"condicion": cond, "desde_cuando": desde, "notas": notas}); st.rerun()
            if st.session_state.antecedentes_medicos_list:
                st.write("**Médicos Registrados:**")
                for i, ant in enumerate(list(st.session_state.antecedentes_medicos_list)):
                    c1,c2=st.columns([4,1]); c1.markdown(f"- **{ant.get('condicion','')}** ({ant.get('desde_cuando','N/A')}) - *{ant.get('notas','')}*")
                    if c2.button("X", key=f"del_am_{i}"): st.session_state.antecedentes_medicos_list.pop(i); st.rerun()
            st.markdown("---")
            st.subheader("Antecedentes Psiquiátricos")
            with st.form("form_ap_psi", clear_on_submit=True):
                c1,c2=st.columns(2); trast=c1.text_input("Trastorno"); desde_p=c2.date_input("Fecha Diag.", value=None, min_value=min_date_allowed); notas_p=st.text_area("Notas")
                if st.form_submit_button("Añadir Ant. Psiquiátrico"):
                    if trast: st.session_state.antecedentes_psiquiatricos_list.append({"trastorno": trast, "desde_cuando": desde_p, "notas": notas_p}); st.rerun()
            if st.session_state.antecedentes_psiquiatricos_list:
                st.write("**Psiquiátricos Registrados:**")
                for i, ant in enumerate(list(st.session_state.antecedentes_psiquiatricos_list)):
                    c1,c2=st.columns([4,1]); c1.markdown(f"- **{ant.get('trastorno','')}** ({ant.get('desde_cuando','N/A')}) - *{ant.get('notas','')}*")
                    if c2.button("X", key=f"del_ap_{i}"): st.session_state.antecedentes_psiquiatricos_list.pop(i); st.rerun()
            if st.button("💾 Guardar Antecedentes Personales", key="save_antecedentes_personales", use_container_width=True):
                with st.spinner("Guardando..."):
                    antecedentes_personales_data = { 'id_paciente': patient_id, 'antecedentes_medicos': st.session_state.antecedentes_medicos_list, 'antecedentes_psiquiatricos': st.session_state.antecedentes_psiquiatricos_list }
                    if upsert_data('antecedentes_personales', antecedentes_personales_data):
                        st.success("✅ Antecedentes Personales guardados.")
                        st.cache_data.clear()
                        st.session_state.loaded_patient_id = None
                        time.sleep(1); st.rerun()

        with st.expander("5. Antecedentes Familiares"):
            with st.form("form_af", clear_on_submit=True):
                c1,c2,c3 = st.columns(3)
                relacion = c1.selectbox("Relación", ['', 'Padre', 'Madre', 'Hermano/a', 'Otro'], key="af_relacion")
                estado = c2.selectbox("Estado Vital", ['Vivo', 'Fallecido'])
                edad = c3.number_input("Edad", min_value=0, step=1, value=0)
                otra_relacion = st.text_input("Especifique otra relación", key="af_otra_relacion") if st.session_state.get('af_relacion') == 'Otro' else ''
                condicion = st.text_input("Condición Médica Principal", key="af_condicion")
                narrativa = st.text_area("Narrativa Psiquiátrica Familiar", key="af_narrativa")
                if st.form_submit_button("Añadir Familiar"):
                    if relacion:
                        condiciones = [{"condicion": condicion, "otra_condicion": "", "notas": ""}] if condicion else []
                        st.session_state.familiares_list.append({ "relacion": relacion, "otra_relacion": otra_relacion, "estado_vital": estado, "edad": int(edad), "condiciones_medicas": condiciones, "narrativa_psiquiatrica": narrativa })
                        st.rerun()
            if st.session_state.familiares_list:
                st.subheader("Familiares Registrados")
                for i, f in enumerate(list(st.session_state.familiares_list)):
                    rel = f.get('otra_relacion') or f.get('relacion'); cond = "N/A"
                    if f.get('condiciones_medicas') and f['condiciones_medicas']: cond = f['condiciones_medicas'][0].get('condicion', 'N/A')
                    c1,c2 = st.columns([5,1]); c1.markdown(f"- **{rel}** ({f.get('edad')} años, {f.get('estado_vital')}) - **Condición:** {cond}")
                    if c2.button("X", key=f"del_fam_{i}"): st.session_state.familiares_list.pop(i); st.rerun()
            if st.button("💾 Guardar Antecedentes Familiares", key="save_antecedentes_familiares", use_container_width=True):
                with st.spinner("Guardando..."):
                    antecedentes_familiares_data = { 'id_paciente': patient_id, 'familiares': st.session_state.familiares_list }
                    if upsert_data('antecedentes_familiares', antecedentes_familiares_data):
                        st.success("✅ Antecedentes Familiares guardados.")
                        st.cache_data.clear()
                        st.session_state.loaded_patient_id = None
                        time.sleep(1); st.rerun()

        with st.expander("6. Hábitos Psicobiológicos"):
            data = st.session_state.get('habitos_psicobiologicos', {})
            sueno = data.get('sueno', {}); cafeina = data.get('cafeina', {}); alcohol = data.get('alcohol', {}); sustancias = data.get('sustancias_ilicitas', {})
            st.checkbox("Presenta insomnio", key="hp_sueno_insomnio", value=sueno.get('insomnio', False))
            if st.session_state.hp_sueno_insomnio:
                s_opts = ['', 'Insomnio de conciliacion', 'Insomnio de mantenimiento', 'Insomnio mixto']; st.selectbox("Tipo de insomnio", s_opts, key="hp_sueno_tipo_insomnio", index=s_opts.index(sueno.get('tipo_insomnio')) if sueno.get('tipo_insomnio') in s_opts else 0)
                c1,c2 = st.columns(2)
                c1.number_input("Horas de sueño", 0.0, 24.0, step=0.5, key="hp_sueno_horas", value=float(sueno.get('horas', 0.0)))
                c2.date_input("Insomnio desde", key="hp_sueno_desde_cuando_insomnio", value=safe_date_converter(sueno.get('desde_cuando_insomnio')), min_value=min_date_allowed)
            st.divider()
            st.checkbox("Consume cafeína", key="hp_cafeina_consume", value=cafeina.get('consume', False))
            if st.session_state.hp_cafeina_consume:
                c1,c2,c3=st.columns(3)
                c1.text_input("Cantidad", key="hp_cafeina_cantidad", value=cafeina.get('cantidad',''))
                c2.text_input("Frecuencia", key="hp_cafeina_frecuencia",value=cafeina.get('frecuencia',''))
                c3.date_input("Consume desde", key="hp_cafeina_desde_cuando", value=safe_date_converter(cafeina.get('desde_cuando')), min_value=min_date_allowed)
            st.divider()
            st.checkbox("Consume alcohol", key="hp_alcohol_consume", value=alcohol.get('consume', False))
            if st.session_state.hp_alcohol_consume:
                c1,c2,c3=st.columns(3)
                c1.text_input("Frecuencia", key="hp_alcohol_frecuencia", value=alcohol.get('frecuencia',''))
                c2.text_input("Patrón", key="hp_alcohol_patron", value=alcohol.get('patron',''))
                c3.date_input("Consume desde", key="hp_alcohol_desde_cuando", value=safe_date_converter(alcohol.get('desde_cuando')), min_value=min_date_allowed)
            st.divider()
            st.checkbox("Consume sustancias ilícitas", key="hp_sustancias_consume", value=sustancias.get('consume', False))
            if st.session_state.hp_sustancias_consume:
                c1,c2,c3=st.columns(3)
                c1.text_input("Tipo", key="hp_sustancias_tipo", value=sustancias.get('tipo',''))
                c2.text_input("Frecuencia", key="hp_sustancias_frecuencia",value=sustancias.get('frecuencia',''))
                c3.date_input("Consume desde", key="hp_sustancias_desde_cuando", value=safe_date_converter(sustancias.get('desde_cuando')), min_value=min_date_allowed)
            if st.button("💾 Guardar Hábitos Psicobiológicos", key="save_habitos", use_container_width=True):
                with st.spinner("Guardando..."):
                    habitos_data = {'id_paciente': patient_id, 'sueno': {'insomnio': st.session_state.hp_sueno_insomnio, 'tipo_insomnio': st.session_state.get('hp_sueno_tipo_insomnio'), 'horas': st.session_state.get('hp_sueno_horas'), 'desde_cuando_insomnio': safe_date_converter(st.session_state.get('hp_sueno_desde_cuando_insomnio'))}, 'cafeina': {'consume': st.session_state.hp_cafeina_consume, 'cantidad': st.session_state.get('hp_cafeina_cantidad'), 'frecuencia': st.session_state.get('hp_cafeina_frecuencia'), 'desde_cuando': safe_date_converter(st.session_state.get('hp_cafeina_desde_cuando'))}, 'alcohol': {'consume': st.session_state.hp_alcohol_consume, 'frecuencia': st.session_state.get('hp_alcohol_frecuencia'), 'patron': st.session_state.get('hp_alcohol_patron'), 'desde_cuando': safe_date_converter(st.session_state.get('hp_alcohol_desde_cuando'))}, 'sustancias_ilicitas': {'consume': st.session_state.hp_sustancias_consume, 'tipo': st.session_state.get('hp_sustancias_tipo'), 'frecuencia': st.session_state.get('hp_sustancias_frecuencia'), 'desde_cuando': safe_date_converter(st.session_state.get('hp_sustancias_desde_cuando'))}}
                    if upsert_data('habitos_psicobiologicos', habitos_data):
                        st.success("✅ Hábitos Psicobiológicos guardados.")
                        st.cache_data.clear()
                        st.session_state.loaded_patient_id = None
                        time.sleep(1); st.rerun()

        with st.expander("7. Diagnósticos"):
            c1,c2 = st.columns(2)
            diag_cat = c1.selectbox("Categoría", [''] + diag_categories, key="diag_cat_sel")
            subcat_opts = ['']
            if diag_cat: subcat_opts.extend(sorted([k[1] for k in grouped_diag_data if k[0] == diag_cat]))
            diag_subcat = c1.selectbox("Subcategoría", subcat_opts, key="diag_subcat_sel")
            is_diag_selected = bool(diag_subcat)
            spec_options, sev_options = [], ['']
            if is_diag_selected:
                details = grouped_diag_data.get((diag_cat, diag_subcat), [{}])[0]
                spec_str = details.get('Especificadores', ''); sev_str = details.get('Nivel_Gravedad', '')
                if spec_str: spec_options = [s.strip() for s in spec_str.split(';') if s.strip()]
                if sev_str: sev_options.extend([s.strip() for s in sev_str.split('/') if s.strip()])
            c2.multiselect("Especificadores", spec_options, key="diag_spec_sel", disabled=not is_diag_selected)
            c2.selectbox("Gravedad", sev_options, key="diag_sev_sel", disabled=not is_diag_selected)
            if st.button("Añadir Diagnóstico"):
                if diag_cat and diag_subcat:
                    st.session_state.current_patient_diagnoses.append({"categoria": diag_cat, "subcategoria": diag_subcat, "especificadores": st.session_state.diag_spec_sel, "gravedad": st.session_state.diag_sev_sel}); st.rerun()
            if st.session_state.current_patient_diagnoses:
                st.markdown("---"); st.subheader("Diagnósticos Registrados")
                for i, diag in enumerate(list(st.session_state.current_patient_diagnoses)):
                    c1,c2 = st.columns([5,1]); summary = f"**{diag['subcategoria']}**"; spec = ", ".join(diag.get('especificadores',[])); summary += f" (*{spec}*)" if spec else ""; summary += f" - {diag.get('gravedad')}" if diag.get('gravedad') else ""
                    c1.markdown(f"- {summary}")
                    if c2.button("X", key=f"del_diag_{i}"): st.session_state.current_patient_diagnoses.pop(i); st.rerun()
            if st.button("💾 Guardar Diagnósticos", key="save_diagnosticos", use_container_width=True):
                with st.spinner("Guardando..."):
                    diag_list = [{'categoria': d.get('categoria'), 'subcategoria': d.get('subcategoria'), 'especificadores': ', '.join(d.get('especificadores', [])) if isinstance(d.get('especificadores'), list) else d.get('especificadores'), 'gravedad': d.get('gravedad')} for d in st.session_state.current_patient_diagnoses]
                    if upsert_data('diagnosticos', {'id_paciente': patient_id, 'diagnosticos': diag_list}):
                        st.success("✅ Diagnósticos guardados.")
                        st.cache_data.clear()
                        st.session_state.loaded_patient_id = None
                        time.sleep(1); st.rerun()

        with st.expander("8. Tratamientos"):
            st.subheader("Tratamiento Farmacológico")
            with st.form("form_trat_farma", clear_on_submit=True):
                c1,c2,c3 = st.columns(3); farmaco = c1.text_input("Fármaco"); dosis = c2.text_input("Dosis"); via = c3.text_input("Vía")
                c4,c5 = st.columns(2); inicio = c4.date_input("Fecha Inicio", value=None, min_value=min_date_allowed); actualizacion = c5.date_input("Última Actualización", value=datetime.date.today(), min_value=min_date_allowed)
                notas = st.text_area("Notas")
                if st.form_submit_button("Añadir Tto. Farmacológico"):
                    if farmaco: st.session_state.tratamientos_farmacologicos_list.append({'farmaco':farmaco, 'dosis':dosis, 'via':via, 'inicio':inicio, 'ultima_actualizacion':actualizacion, 'notas':notas}); st.rerun()
            if st.session_state.tratamientos_farmacologicos_list: st.dataframe(pd.DataFrame(st.session_state.tratamientos_farmacologicos_list), use_container_width=True)
            st.markdown("---")
            st.subheader("Tratamiento Psiquiátrico")
            with st.form("form_trat_psi", clear_on_submit=True):
                c1,c2,c3 = st.columns(3); farmaco_p = c1.text_input("Fármaco"); dosis_p = c2.text_input("Dosis"); via_p = c3.text_input("Vía")
                c4,c5 = st.columns(2); inicio_p = c4.date_input("Fecha Inicio", value=None, min_value=min_date_allowed); actualizacion_p = c5.date_input("Última Actualización", value=datetime.date.today(), min_value=min_date_allowed)
                notas_p = st.text_area("Notas")
                if st.form_submit_button("Añadir Tto. Psiquiátrico"):
                    if farmaco_p: st.session_state.tratamientos_psiquiatricos_list.append({'farmaco':farmaco_p, 'dosis':dosis_p, 'via':via_p, 'inicio':inicio_p, 'ultima_actualizacion':actualizacion_p, 'notas':notas_p}); st.rerun()
            if st.session_state.tratamientos_psiquiatricos_list: st.dataframe(pd.DataFrame(st.session_state.tratamientos_psiquiatricos_list), use_container_width=True)
            if st.button("💾 Guardar Tratamientos", key="save_tratamientos", use_container_width=True):
                with st.spinner("Guardando..."):
                    tratamientos_data = { 'id_paciente': patient_id, 'farmacologico': st.session_state.tratamientos_farmacologicos_list, 'psiquiatrico': st.session_state.tratamientos_psiquiatricos_list }
                    if upsert_data('tratamientos', tratamientos_data):
                        st.success("✅ Tratamientos guardados.")
                        st.cache_data.clear()
                        st.session_state.loaded_patient_id = None
                        time.sleep(1); st.rerun()

    with tab2:
        data = st.session_state.get('datos_administrativos', {})
        with st.form("admin_form"):
            c1, c2 = st.columns(2)
            with c1:
                st.text_input("Teléfono de Contacto", data.get('telefono',''), key="admin_telefono")
                frec_opts=['','Semanal','Quincenal','Mensual','Variable']; st.selectbox("Frecuencia de Sesiones", frec_opts, key="admin_frecuencia", index=frec_opts.index(data.get('frecuencia')) if data.get('frecuencia') in frec_opts else 0)
                st.time_input("Hora de Sesión Habitual", safe_time_converter(data.get('hora_sesion')), key="admin_hora_sesion")
                st.number_input("Honorarios (€)", min_value=0.0, step=5.0, value=float(data.get('Honorarios', 0.0)), key="admin_honorarios")
            with c2:
                st.text_input("Correo Electrónico", data.get('email',''), key="admin_email")
                dias_opts=['','Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo']; st.selectbox("Día de Sesión Habitual", dias_opts, key="admin_dia_sesion", index=dias_opts.index(data.get('dia_sesion')) if data.get('dia_sesion') in dias_opts else 0)
                est_opts=['Activo','Inactivo']; st.selectbox("Estado del Paciente", est_opts, key="admin_estado_paciente", index=est_opts.index(data.get('estado_paciente')) if data.get('estado_paciente') in est_opts else 0)
                st.date_input("Próxima Fecha de Pago", safe_date_converter(data.get('Fecha de pago')), key="admin_fecha_pago")

            if st.form_submit_button("💾 Guardar Datos Administrativos", use_container_width=True, type="primary"):
                admin_data_to_save = {
                    'id_paciente': patient_id, 'telefono': st.session_state.admin_telefono, 'email': st.session_state.admin_email,
                    'frecuencia': st.session_state.admin_frecuencia, 'dia_sesion': st.session_state.admin_dia_sesion,
                    'hora_sesion': st.session_state.admin_hora_sesion,
                    'estado_paciente': st.session_state.admin_estado_paciente,
                    'Honorarios': safe_decimal_converter(st.session_state.admin_honorarios),
                    'Fecha de pago': safe_date_converter(st.session_state.admin_fecha_pago)
                }
                with st.spinner("Guardando..."):
                    if upsert_data('datos_administrativos', admin_data_to_save):
                        st.success("✅ Datos administrativos guardados.");
                        st.cache_data.clear()
                        st.session_state.loaded_patient_id = None
                        time.sleep(1); st.rerun()
                    else: st.error("❌ Error al guardar datos administrativos.")

    with tab_sesiones:
        with st.form("form_add_sesion", clear_on_submit=True):
            st.write("**Añadir Nueva Sesión**")
            fecha_sesion = st.date_input("Fecha de Sesión", value=datetime.date.today())
            transcripcion_content = st_quill(key="sesion_transcripcion_quill")
            if st.form_submit_button("Añadir Sesión"):
                if fecha_sesion and transcripcion_content:
                    sesion_data = {'id_sesion': generate_unique_id(), 'id_paciente': patient_id, 'fecha_de_sesion': fecha_sesion, 'transcripcion': transcripcion_content}
                    if upsert_data('sesiones', sesion_data):
                        st.success("✅ Sesión añadida."); st.session_state.sesiones_list.append(sesion_data)
                        st.cache_data.clear()
                        st.session_state.loaded_patient_id = None
                        st.rerun()
        st.divider()
        if st.session_state.sesiones_list:
            st.write("**Sesiones Registradas**")
            @st.dialog("Confirmar Eliminación de Sesión")
            def delete_session_dialog(session_id, session_date):
                st.warning(f"¿Seguro que quieres eliminar la sesión del **{session_date}**?")
                c1, c2 = st.columns(2)
                if c1.button("Cancelar", use_container_width=True): st.rerun()
                if c2.button("Eliminar", type="primary", use_container_width=True):
                    if delete_record('sesiones', session_id, 'id_sesion'):
                        st.session_state.sesiones_list = [s for s in st.session_state.sesiones_list if s.get('id_sesion') != session_id]
                        st.cache_data.clear()
                        st.rerun()

            for sesion in sorted(st.session_state.sesiones_list, key=lambda x: (safe_date_converter(x.get('fecha_de_sesion')) or datetime.date.min), reverse=True):
                with st.expander(f"**Sesión del {sesion.get('fecha_de_sesion')}**"):
                    st.markdown(sesion.get('transcripcion', ''), unsafe_allow_html=True)
                    if st.button("Eliminar Sesión", key=f"del_sesion_{sesion.get('id_sesion')}", type="secondary"):
                        delete_session_dialog(sesion.get('id_sesion'), sesion.get('fecha_de_sesion'))
        else: st.info("No hay sesiones registradas.")

    with tab5:
        with st.form("form_add_cita", clear_on_submit=True):
            st.write("**Añadir Nueva Cita**")
            c1, c2 = st.columns(2)
            default_datetime = datetime.datetime.now()
            fecha_cita = c1.date_input("Fecha de Cita", value=default_datetime.date())
            hora_cita = c2.time_input("Hora de Cita", value=default_datetime.time())
            est_cita_opts = ['Pendiente','Confirmada','Cancelada','Realizada']
            estado_cita = st.selectbox("Estado de la Cita", est_cita_opts, key="cita_estado")
            if st.form_submit_button("Añadir Cita"):
                cita_data = {'id_cita': generate_unique_id(), 'id_paciente': patient_id, 'fecha_de_cita': fecha_cita, 'hora': hora_cita, 'estado_de_la_cita': estado_cita}
                if upsert_data('agenda', cita_data):
                    st.success("✅ Cita añadida."); st.session_state.agenda_list.append(cita_data)
                    st.cache_data.clear()
                    st.rerun()
        st.divider()
        st.write("**Citas Registradas**")
        if st.session_state.agenda_list:
            @st.dialog("Confirmar Eliminación de Cita")
            def delete_appointment_dialog(cita_id, cita_date, cita_time):
                st.warning(f"¿Seguro que quieres eliminar la cita del **{cita_date}** a las **{cita_time}**?")
                c1, c2 = st.columns(2)
                if c1.button("Cancelar", use_container_width=True): st.rerun()
                if c2.button("Eliminar", type="primary", use_container_width=True):
                    if delete_record('agenda', cita_id, 'id_cita'):
                        st.session_state.agenda_list = [c for c in st.session_state.agenda_list if c.get('id_cita') != cita_id]
                        st.cache_data.clear()
                        st.rerun()

            for cita in sorted(st.session_state.agenda_list, key=lambda x: (safe_date_converter(x.get('fecha_de_cita')) or datetime.date.min, safe_time_converter(x.get('hora')) or datetime.time.min)):
                col1, col2 = st.columns([4, 1])
                col1.markdown(f"- **{safe_date_converter(cita.get('fecha_de_cita'))}** a las {safe_time_converter(cita.get('hora'))} - Estado: **{cita.get('estado_de_la_cita')}**")
                if col2.button("Eliminar", key=f"del_cita_{cita.get('id_cita')}"):
                    delete_appointment_dialog(cita.get('id_cita'), cita.get('fecha_de_cita'), cita.get('hora'))
        else: st.info("No hay citas registradas para este paciente.")

    with tab3:
        data = st.session_state.get('comprension_psicodinamica', {})
        st.write("**Comprensión Psicodinámica**")
        comprension_content = st_quill(value=data.get('comprension', ''), key="comprension_quill")
        if st.button("💾 Guardar Comprensión", use_container_width=True, type="primary"):
            if upsert_data('comprension_psicodinamica', {'id_paciente': patient_id, 'comprension': comprension_content}):
                st.success("✅ Comprensión guardada.")
                st.cache_data.clear()
                st.session_state.loaded_patient_id = None
                time.sleep(1); st.rerun()
            else: st.error("❌ Error al guardar la comprensión.")

    with tab4:
        @st.dialog("Resumen de Historia Clínica", width="large")
        def show_summary_dialog():
            st.header(f"Informe de: {patient_name}")
            st.caption(f"Generado el: {datetime.date.today().strftime('%d/%m/%Y')}")
            st.divider()

            st.subheader("1. Datos de Identificación")
            id_data = st.session_state.get('datos_identificacion', {})
            admin_data = st.session_state.get('datos_administrativos', {})
            col1, col2 = st.columns(2)
            col1.markdown(f"**Edad:** {id_data.get('edad', 'N/A')} años")
            col1.markdown(f"**Género:** {id_data.get('genero', 'N/A')}")
            col1.markdown(f"**Estado Civil:** {id_data.get('estado_civil', 'N/A')}")
            col1.markdown(f"**Teléfono:** {admin_data.get('telefono', 'N/A')}")
            col2.markdown(f"**Ocupación:** {id_data.get('posicion_ocupacion', 'N/A')} ({id_data.get('estado_laboral', 'N/A')})")
            col2.markdown(f"**Nivel Educativo:** {id_data.get('nivel_educativo', 'N/A')}")
            col2.markdown(f"**País de Procedencia:** {id_data.get('pais_procedencia', 'N/A')}")
            col2.markdown(f"**Email:** {admin_data.get('email', 'N/A')}")
            if st.session_state.hijos_list:
                nombres_hijos = ", ".join([f"{h['nombre']} ({h['edad']} años)" for h in st.session_state.hijos_list])
                st.markdown(f"**Hijos:** {nombres_hijos}")

            st.subheader("2. Motivo de Consulta y Enfermedad Actual")
            st.markdown("**Motivo:**")
            st.markdown(st.session_state.get('motivo_de_consulta',{}).get('motivo_de_consulta', 'No registrado.'), unsafe_allow_html=True)
            with st.expander("Ver Enfermedad Actual"):
                st.markdown(st.session_state.get('enfermedad_actual',{}).get('enfermedad_actual', 'No registrada.'), unsafe_allow_html=True)

            st.subheader("3. Antecedentes")
            with st.expander("Personales"):
                st.markdown("**Médicos:**")
                if st.session_state.antecedentes_medicos_list:
                    for ant in st.session_state.antecedentes_medicos_list: st.markdown(f"- {ant.get('condicion','')} ({ant.get('desde_cuando','N/A')})")
                else: st.caption("No registrados.")
                st.markdown("**Psiquiátricos:**")
                if st.session_state.antecedentes_psiquiatricos_list:
                    for ant in st.session_state.antecedentes_psiquiatricos_list: st.markdown(f"- {ant.get('trastorno','')} ({ant.get('desde_cuando','N/A')})")
                else: st.caption("No registrados.")
            with st.expander("Familiares"):
                 if st.session_state.familiares_list:
                    for f in st.session_state.familiares_list:
                        rel = f.get('otra_relacion') or f.get('relacion')
                        cond = "N/A"
                        if f.get('condiciones_medicas') and f['condiciones_medicas']: cond = f['condiciones_medicas'][0].get('condicion', 'N/A')
                        st.markdown(f"- **{rel}** ({f.get('estado_vital')}): {cond}")
                 else: st.caption("No registrados.")

            st.subheader("4. Examen Mental")
            examen_mental_data = st.session_state.get('examen_mental', {})
            narrativa = examen_mental_data.get('observaciones_generales')
            if narrativa:
                st.markdown(narrativa, unsafe_allow_html=True)
            else:
                st.caption("No se registró información del examen mental.")

            st.subheader("5. Diagnósticos")
            if st.session_state.current_patient_diagnoses:
                for diag in st.session_state.current_patient_diagnoses:
                    spec = ", ".join(diag.get('especificadores', []))
                    st.markdown(f"- **{diag.get('subcategoria')}** {f'({spec})' if spec else ''} - *Gravedad: {diag.get('gravedad', 'N/A')}*")
            else: st.caption("No hay diagnósticos registrados.")

            st.subheader("6. Tratamiento Actual")
            if st.session_state.tratamientos_farmacologicos_list or st.session_state.tratamientos_psiquiatricos_list:
                c1, c2 = st.columns(2)
                with c1:
                    st.markdown("**Farmacológico:**")
                    if st.session_state.tratamientos_farmacologicos_list:
                        for t in st.session_state.tratamientos_farmacologicos_list: st.markdown(f"- {t.get('farmaco')} {t.get('dosis')}")
                    else: st.caption("No registrado.")
                with c2:
                    st.markdown("**Psiquiátrico:**")
                    if st.session_state.tratamientos_psiquiatricos_list:
                        for t in st.session_state.tratamientos_psiquiatricos_list: st.markdown(f"- {t.get('farmaco')} {t.get('dosis')}")
                    else: st.caption("No registrado.")
            else: st.caption("No hay tratamientos registrados.")

            st.divider()
            if st.button("Cerrar", key="close_summary"): st.rerun()

        st.info("Genera una vista consolidada y detallada de los datos más importantes del paciente.")
        if st.button("Ver Resumen de Historia Clínica", use_container_width=True, type="primary"):
            show_summary_dialog()

elif st.session_state.page == 'agenda':
    st.markdown('<h1 class="page-title">🗓️ Agenda General</h1>', unsafe_allow_html=True)
    @st.cache_data(ttl=60)
    def get_global_agenda(_client):
        if not _client: return pd.DataFrame()
        query = f"""SELECT p.nombre_completo, a.fecha_de_cita, a.hora, a.estado_de_la_cita FROM `{PROJECT_ID}.{DATASET_ID}.agenda` AS a LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.pacientes` AS p ON a.id_paciente = p.id_paciente WHERE a.fecha_de_cita >= CURRENT_DATE() ORDER BY a.fecha_de_cita, a.hora"""
        try: return _client.query(query).to_dataframe()
        except Exception as e: st.error(f"Error al cargar agenda: {e}"); return pd.DataFrame()
    agenda_df = get_global_agenda(client)
    if not agenda_df.empty: st.dataframe(agenda_df.rename(columns={'nombre_completo': 'Paciente', 'fecha_de_cita': 'Fecha', 'hora': 'Hora', 'estado_de_la_cita': 'Estado'}), use_container_width=True)
    else: st.info("No hay citas próximas en la agenda.")

elif st.session_state.page == 'resumen':
    st.markdown('<h1 class="page-title">📊 Resumen Administrativo</h1>', unsafe_allow_html=True)
    @st.cache_data(ttl=600)
    def get_dashboard_kpis(_client):
        if not _client: return {}
        kpis = {}
        try:
            query_income = f"""SELECT SUM(IF(EXTRACT(YEAR FROM `Fecha de pago`) = EXTRACT(YEAR FROM CURRENT_DATE()) AND EXTRACT(MONTH FROM `Fecha de pago`) = EXTRACT(MONTH FROM CURRENT_DATE()), Honorarios, 0)) AS current_month_income FROM `{PROJECT_ID}.{DATASET_ID}.datos_administrativos`"""
            df_income = _client.query(query_income).to_dataframe()
            kpis['income'] = df_income['current_month_income'].iloc[0] if pd.notna(df_income['current_month_income'].iloc[0]) else 0

            query_patients = f"""SELECT d.estado_paciente, i.genero, COUNT(p.id_paciente) as count FROM `{PROJECT_ID}.{DATASET_ID}.pacientes` p LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.datos_administrativos` d ON p.id_paciente = d.id_paciente LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.datos_identificacion` i ON p.id_paciente = i.id_paciente GROUP BY 1, 2"""
            df_patients = _client.query(query_patients).to_dataframe()
            kpis['total_patients'] = int(df_patients['count'].sum())
            kpis['active_patients'] = int(df_patients[df_patients['estado_paciente'] == 'Activo']['count'].sum())
            kpis['inactive_patients'] = kpis['total_patients'] - kpis['active_patients']
            kpis['gender_dist'] = df_patients.groupby('genero')['count'].sum().to_dict()
            return kpis
        except Exception as e: st.warning(f"No se pudo calcular KPIs: {e}"); return {}

    kpis = get_dashboard_kpis(client)
    c1, c2, c3 = st.columns(3)
    c1.metric("Ingresos del Mes", f"{kpis.get('income', 0):,.2f} €")
    c2.metric("Total Pacientes", kpis.get('total_patients', 0))
    c3.metric("Pacientes Activos", f"{kpis.get('active_patients',0)} (Inactivos: {kpis.get('inactive_patients', 0)})")

    gender_data = kpis.get('gender_dist', {})
    if gender_data:
        st.markdown("**Distribución por Género:**")
        clean_gender_data = {k: v for k, v in gender_data.items() if k}
        if clean_gender_data:
            df_gender = pd.DataFrame.from_dict(clean_gender_data, orient='index', columns=['Pacientes'])
            st.bar_chart(df_gender)

else:
    st.session_state.page = 'consultorio'
    st.rerun()


Overwriting app.py


#FASE DE EJECUCIÓN

# Ejecución de la Aplicación Streamlit y del Túnel ngrok

In [10]:
# --- PASO 5: EJECUCIÓN DE STREAMLIT Y NGROK ---
import time
from pyngrok import ngrok
import getpass
import os
import sys

# Detener cualquier proceso de Streamlit o ngrok que use el puerto 8501
print("Intentando detener procesos existentes en puerto 8501...")
try:
    !pkill -f "streamlit run"
    ngrok.kill()
    time.sleep(5)  # Esperar a que los procesos terminen
    print("✓ Procesos detenidos.")
except Exception as e:
    print(f"⚠️ No se encontraron procesos para detener o ocurrió un error: {e}")

print("-" * 50)

# Iniciar la aplicación Streamlit en segundo plano
print("Iniciando aplicación Streamlit en segundo plano...")
!streamlit run app.py --server.port 8501 > streamlit_output.log 2>&1 &
print("✓ Aplicación Streamlit iniciada en segundo plano en el puerto 8501.")
time.sleep(10) # Dar tiempo a que Streamlit inicie

print("-" * 50)

# Iniciar el túnel ngrok y obtener la URL pública
print("Iniciando túnel ngrok...")
try:
    tunnel = ngrok.connect(8501)
    public_url = tunnel.public_url
    print("\n🎉 ¡Éxito! Túnel ngrok establecido. URL pública:")
    print(f"🔗 {public_url}")
    print("\n👉 Haz clic en la URL de arriba para acceder a tu aplicación Streamlit.")
    print("Nota: La aplicación se está ejecutando en segundo plano. Para detenerla, ejecuta la primera celda de este paso de nuevo.")
except Exception as e:
    print(f"\n⚠️ Ocurrió un error al iniciar el túnel ngrok: {e}")
    print("Por favor, verifica el token de ngrok y el contenido del log de Streamlit.")
    print("\n--- Contenido reciente de streamlit_output.log ---")
    try:
        with open('streamlit_output.log', 'r') as f:
            lines = f.readlines()
            last_lines = lines[-20:] if len(lines) > 20 else lines
            print("".join(last_lines))
    except FileNotFoundError:
        print("El archivo streamlit_output.log aún no se ha creado o encontrado.")
    except Exception as read_e:
        print(f"Ocurrió un error al leer el archivo de log: {read_e}")

print("-" * 50)

Intentando detener procesos existentes en puerto 8501...
✓ Procesos detenidos.
--------------------------------------------------
Iniciando aplicación Streamlit en segundo plano...
✓ Aplicación Streamlit iniciada en segundo plano en el puerto 8501.
--------------------------------------------------
Iniciando túnel ngrok...

🎉 ¡Éxito! Túnel ngrok establecido. URL pública:
🔗 https://43b68039b057.ngrok-free.app

👉 Haz clic en la URL de arriba para acceder a tu aplicación Streamlit.
Nota: La aplicación se está ejecutando en segundo plano. Para detenerla, ejecuta la primera celda de este paso de nuevo.
--------------------------------------------------
